In [1]:
import ee
ee.Authenticate()
ee.Initialize() 

In [2]:
# --- PARTE 0: Parámetros generales ---
from pathlib import Path

# Rutas a tus KMZ (en la carpeta actual o ajusta paths absolutos)
KMZ_FILES = [
    "Corridor_Study_puntos.kmz"
]

# Nombre de salida
OUT_DIR = Path("salidas_aoi")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# CRS proyectado en METROS (Colombia). Ajusta si usas otro oficial de tu proyecto.
CRS_METROS = "EPSG:3116"   # MAGNA-SIRGAS / Colombia Bogota

# Parámetros AOI
BUFFER_M = 300   # radio del buffer por punto (m). Sube/baja según ancho de franja que quieras
SMOOTH_M = 50    # suavizado (buffer negativo para "apretar" bordes tras disolver). 0 = desactivar

print("✓ Parámetros generales cargados")
print(f"Archivos KMZ a procesar: {len(KMZ_FILES)}")
for kmz in KMZ_FILES:
    if Path(kmz).exists():
        print(f"  ✓ {kmz}")
    else:
        print(f"  ⚠️  {kmz} - no encontrado")

✓ Parámetros generales cargados
Archivos KMZ a procesar: 1
  ✓ Corridor_Study_puntos.kmz


In [3]:
# --- PARTE 1: Importar librerías y configurar GEE ---
import ee
import geemap
import pandas as pd
import geopandas as gpd
from pathlib import Path
import zipfile
import tempfile
from xml.etree import ElementTree as ET

# Inicializar Earth Engine
try:
    ee.Initialize()
    print("✓ Google Earth Engine inicializado correctamente")
except Exception as e:
    print(f"Error al inicializar GEE: {e}")
    print("Ejecuta: ee.Authenticate() si es la primera vez")

# Parámetros temporales por década (≥30 años) - ACTUALIZADO
PERIODOS = {
    '1990s': {
        'start': '1990-01-01',
        'end': '1999-12-31',
        'sensor': 'LANDSAT/LT05/C02/T1_L2',
        'nombre': 'Landsat 5 TM (1990s)'
    },
    '2000s': {
        'start': '2000-01-01', 
        'end': '2009-12-31',
        'sensor': 'LANDSAT/LE07/C02/T1_L2',
        'nombre': 'Landsat 7 ETM+ (2000s)'
    },
    '2010s': {
        'start': '2010-01-01',
        'end': '2019-12-31', 
        'sensor': 'LANDSAT/LC08/C02/T1_L2',
        'nombre': 'Landsat 8 OLI (2010s)'
    },
    'Presente_1999-2025': {
        'start': '1999-01-01',  # Período extendido desde 1999
        'end': '2025-12-31',    # Hasta presente (2025)
        'sensor': ['LANDSAT/LC08/C02/T1_L2', 'COPERNICUS/S2_SR_HARMONIZED'],
        'nombre': 'Presente (1999-2025) - Landsat 8 + Sentinel-2'
    }
}

# Parámetros de filtrado
CLOUD_COVER_MAX = 30  # Máximo % nubes
EPOCA_SECA = [12, 1, 2, 3]  # Meses época seca (Dic-Mar)

# Parámetros de escala - DOCUMENTACIÓN EXPLÍCITA
ESCALA_COMPARABILIDAD = 30  # Metros - Forzado para comparabilidad inter-sensor (Landsat estándar)
# NOTA: Sentinel-2 nativo 10m, pero remuestreado a 30m para consistencia temporal
# Para análisis de alta resolución S2, usar escala 10m y documentar explícitamente

print("✓ Configuración cargada")
print(f"Períodos definidos: {list(PERIODOS.keys())}")
print(f"Cobertura temporal: {PERIODOS['1990s']['start']} a {PERIODOS['Presente_1999-2025']['end']}")
print(f"Período extendido: {PERIODOS['Presente_1999-2025']['start']} a {PERIODOS['Presente_1999-2025']['end']}")
print(f"Escala de análisis: {ESCALA_COMPARABILIDAD}m (comparabilidad inter-sensor)")

✓ Google Earth Engine inicializado correctamente
✓ Configuración cargada
Períodos definidos: ['1990s', '2000s', '2010s', 'Presente_1999-2025']
Cobertura temporal: 1990-01-01 a 2025-12-31
Período extendido: 1999-01-01 a 2025-12-31
Escala de análisis: 30m (comparabilidad inter-sensor)


## SECCIÓN 2: Importar Áreas de Interés (AOI) desde KMZ

Los archivos KMZ contienen datos vectoriales de los puntos críticos. Necesitamos extraer las geometrías y convertirlas a formato compatible con GEE.

In [4]:
# --- PARTE 2: Funciones para importar KMZ ---

def extraer_coordenadas_kml(kml_content):
    """
    Extrae coordenadas de un archivo KML parseando el XML
    """
    try:
        root = ET.fromstring(kml_content)
        
        # Namespaces comunes en KML
        ns = {'kml': 'http://www.opengis.net/kml/2.2'}
        
        coordenadas = []
        nombres = []
        
        # Buscar Placemarks
        for placemark in root.findall('.//kml:Placemark', ns):
            nombre_elem = placemark.find('kml:name', ns)
            nombre = nombre_elem.text if nombre_elem is not None else "Sin_nombre"
            
            # Buscar coordenadas en Point
            point = placemark.find('.//kml:Point/kml:coordinates', ns)
            if point is not None:
                coords_text = point.text.strip()
                # Formato: lon,lat,alt
                lon, lat = coords_text.split(',')[:2]
                coordenadas.append([float(lon), float(lat)])
                nombres.append(nombre)
                continue
            
            # Buscar coordenadas en Polygon
            polygon = placemark.find('.//kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates', ns)
            if polygon is not None:
                coords_text = polygon.text.strip()
                coords_pairs = []
                for coord in coords_text.split():
                    if coord.strip():
                        lon, lat = coord.split(',')[:2]
                        coords_pairs.append([float(lon), float(lat)])
                coordenadas.append(coords_pairs)
                nombres.append(nombre)
        
        return coordenadas, nombres
    
    except Exception as e:
        print(f"Error procesando KML: {e}")
        return [], []
def kmz_a_ee_layers(kmz_files, buffer_metros=300):
    """
    Devuelve (puntos_fc, aois_fc): puntos originales + buffers/polígonos
    """
    puntos, nombres_puntos = [], []
    geoms_aois, nombres_aois = [], []

    for kmz_file in kmz_files:
        if not Path(kmz_file).exists():
            print(f"  ⚠️  Archivo no encontrado: {kmz_file}")
            continue

        try:
            with zipfile.ZipFile(kmz_file, 'r') as zip_ref:
                kml_files = [f for f in zip_ref.namelist() if f.endswith('.kml')]
                if not kml_files:
                    print(f"  ⚠️  No se encontró KML en {kmz_file}")
                    continue
                with zip_ref.open(kml_files[0]) as kml_file:
                    kml_content = kml_file.read().decode('utf-8')

            coords_list, nombres = extraer_coordenadas_kml(kml_content)
            print(f"  ✓ Extraídas {len(coords_list)} geometrías")

            for i, coords in enumerate(coords_list):
                nombre = nombres[i] if i < len(nombres) else f"Punto_{i+1}"

                if isinstance(coords[0], list):
                    # Polígono
                    poly = ee.Geometry.Polygon(coords)
                    geoms_aois.append(poly)
                    nombres_aois.append(nombre)
                else:
                    # Punto
                    pt = ee.Geometry.Point(coords)
                    puntos.append(pt)
                    nombres_puntos.append(nombre)
                    # Buffer como AOI
                    geoms_aois.append(pt.buffer(buffer_metros))
                    nombres_aois.append(nombre)

        except Exception as e:
            print(f"  ❌ Error procesando {kmz_file}: {e}")

    puntos_fc = ee.FeatureCollection([
        ee.Feature(geom, {'nombre': nombres_puntos[i], 'tipo': 'punto'})
        for i, geom in enumerate(puntos)
    ]) if puntos else ee.FeatureCollection([])

    aois_fc = ee.FeatureCollection([
        ee.Feature(geom, {'nombre': nombres_aois[i], 'tipo': 'aoi', 'buffer_m': buffer_metros})
        for i, geom in enumerate(geoms_aois)
    ]) if geoms_aois else ee.FeatureCollection([])

    return puntos_fc, aois_fc
print("✓ Funciones de importación KMZ definidas")

✓ Funciones de importación KMZ definidas


In [5]:
# --- EJECUTAR: Importar AOI desde archivos KMZ ---

# Convertir archivos KMZ a geometrías de Earth Engine
print("=== IMPORTANDO ÁREAS DE INTERÉS ===")
puntos_fc, aois = kmz_a_ee_layers(KMZ_FILES, buffer_metros=BUFFER_M)

if aois is None or aois.size().getInfo() == 0:
    raise RuntimeError("❌ No se pudieron extraer AOIs")
print(f"✓ AOI creadas: {aois.size().getInfo()} | Puntos: {puntos_fc.size().getInfo()}")

aoi_bounds = aois.geometry().bounds()

if aois is not None:
    print(f"✓ AOI creadas exitosamente")
    print(f"Número total de áreas: {aois.size().getInfo()}")
    
    # Obtener información de las AOI
    aoi_info = aois.getInfo()
    print("\nÁreas importadas:")
    for i, feature in enumerate(aoi_info['features']):
        nombre = feature['properties']['nombre']
        print(f"  {i+1}. {nombre}")
    
    # Calcular extensión total para centrar mapa
    bounds = aois.geometry().bounds()
    print(f"\n✓ AOI listas para análisis multitemporal")
    
else:
    print("❌ Error al importar AOI. Verifica los archivos KMZ.")
    # Crear AOI de ejemplo si falla la importación
    print("Creando AOI de ejemplo para continuar...")
    ejemplo_coords = [[-75.5, 6.2], [-75.4, 6.2], [-75.4, 6.3], [-75.5, 6.3], [-75.5, 6.2]]
    aoi_ejemplo = ee.Geometry.Polygon([ejemplo_coords])
    aois = ee.FeatureCollection([
        ee.Feature(aoi_ejemplo, {'nombre': 'AOI_Ejemplo', 'sitio': 'Region_A'})
    ])

=== IMPORTANDO ÁREAS DE INTERÉS ===
  ✓ Extraídas 24 geometrías
✓ AOI creadas: 24 | Puntos: 24
✓ AOI creadas exitosamente
Número total de áreas: 24

Áreas importadas:
  1. Punto 24 (k12+100)
  2. Punto 23 (k12+040)
  3. Punto 22 (k11+950)
  4. Punto 21  (k11+710)
  5. Punto 20 (k11+600)
  6. Punto 18 (k11+330)
  7. Punto 17 (k11+150)
  8. Punto 16 (k10+850)
  9. Punto 15 (k10+730)
  10. Punto 14 (k10+490)
  11. Punto 13 (k10+340)
  12. Punto 11 (k8+900)
  13. Punto 12 (k9+100)
  14. Punto 10 (k8+440)
  15. Punto 9 (7+170)
  16. Punto 8 (k6+750)
  17. Punto 7 (K5+730)
  18. Punto 6 (K4+880)
  19. Punto 5 (K3 +115)
  20. Punto 3 (K2+550)
  21. Punto 4 (K1+500)
  22. Punto 2 (K2+330)
  23. Punto 1 (K0+890)
  24. Punto 19 (k11+450)

✓ AOI listas para análisis multitemporal


## SECCIÓN 3: Filtrado de Nubes y Preprocesamiento

Funciones para limpiar imágenes satelitales y aplicar filtros de calidad por sensor.

In [6]:
# --- PARTE 3: Funciones de preprocesamiento ---

def filtrar_nubes_landsat_c2(image):
    """
    Filtro de nubes para Landsat Collection 2 usando QA_PIXEL
    """
    qa = image.select('QA_PIXEL')
    
    # Bits de calidad (Collection 2)
    # Bit 3: Cloud
    # Bit 4: Cloud Shadow  
    # Bit 1: Dilated Cloud
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Sin nubes
    shadow_mask = qa.bitwiseAnd(1 << 4).eq(0)  # Sin sombras
    dilated_mask = qa.bitwiseAnd(1 << 1).eq(0)  # Sin nubes dilatadas
    
    # Combinar máscaras
    mask = cloud_mask.And(shadow_mask).And(dilated_mask)
    
    # Aplicar factor de escala para Surface Reflectance
    scaled = image.multiply(0.0000275).add(-0.2)
    
    return scaled.updateMask(mask).copyProperties(image, ['system:time_start', 'system:index'])

def filtrar_nubes_sentinel2(image):
    """
    Filtro de nubes para Sentinel-2 usando QA60
    """
    qa = image.select('QA60')
    
    # Bit 10: Opaque clouds
    # Bit 11: Cirrus clouds
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)
    cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)
    
    mask = cloud_mask.And(cirrus_mask)
    
    # Aplicar factor de escala
    scaled = image.multiply(0.0001)
    
    return scaled.updateMask(mask).copyProperties(image, ['system:time_start', 'system:index'])

def calcular_indices_landsat(image):
    """
    Calcula índices espectrales para Landsat
    """
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    nbr = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')
    rgb = image.select(['SR_B4', 'SR_B3', 'SR_B2']).rename(['Red', 'Green', 'Blue'])
    return image.addBands([ndvi, nbr, rgb])

def calcular_indices_sentinel2(image):
    """
    Calcula índices espectrales para Sentinel-2
    """
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    nbr = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    rgb = image.select(['B4', 'B3', 'B2']).rename(['Red', 'Green', 'Blue'])
    return image.addBands([ndvi, nbr, rgb])

def filtrar_epoca_seca(collection, meses_epoca_seca):
    """
    Filtra colección por meses de época seca
    """
    # Crear filtro para todos los meses de época seca
    month_filter = ee.Filter.calendarRange(
        meses_epoca_seca[0], meses_epoca_seca[-1], 'month'
    )
    
    # Si los meses cruzan el año (ej: [12, 1, 2, 3])
    if meses_epoca_seca[0] > meses_epoca_seca[-1]:
        # Crear dos filtros separados y combinarlos con OR
        filter1 = ee.Filter.calendarRange(meses_epoca_seca[0], 12, 'month')
        filter2 = ee.Filter.calendarRange(1, meses_epoca_seca[-1], 'month')
        month_filter = ee.Filter.Or(filter1, filter2)
    
    return collection.filter(month_filter)

print("✓ Funciones de preprocesamiento definidas")
print("  - Filtrado de nubes Landsat C2")
print("  - Filtrado de nubes Sentinel-2")  
print("  - Cálculo de índices Landsat")
print("  - Cálculo de índices Sentinel-2")
print("  - Filtrado temporal (época seca)")

✓ Funciones de preprocesamiento definidas
  - Filtrado de nubes Landsat C2
  - Filtrado de nubes Sentinel-2
  - Cálculo de índices Landsat
  - Cálculo de índices Sentinel-2
  - Filtrado temporal (época seca)


## SECCIÓN 4: Colecciones Landsat por Década (1990s-2010s)

Procesamiento de imágenes Landsat para crear composiciones medianas por década.

In [7]:
# --- PARTE 4: Procesamiento Landsat por décadas ---

def procesar_landsat_decada(periodo_key, aoi_geometry):
    """
    Procesa una década específica de Landsat
    """
    periodo = PERIODOS[periodo_key]
    print(f"\n=== PROCESANDO {periodo['nombre']} ===")
    
    # Crear colección base
    collection = ee.ImageCollection(periodo['sensor']) \
        .filterDate(periodo['start'], periodo['end']) \
        .filterBounds(aoi_geometry) \
        .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER_MAX))
    
    print(f"Imágenes antes de filtros: {collection.size().getInfo()}")
    
    # Filtrar por época seca
    collection_epoca_seca = filtrar_epoca_seca(collection, EPOCA_SECA)
    print(f"Imágenes época seca: {collection_epoca_seca.size().getInfo()}")
    
    # Aplicar filtros de nubes y calcular índices (solo Landsat en esta función)
    collection_procesada = collection_epoca_seca \
        .map(filtrar_nubes_landsat_c2) \
        .map(calcular_indices_landsat)
    
    print(f"Imágenes procesadas: {collection_procesada.size().getInfo()}")
    
    # Crear composición mediana
    if collection_procesada.size().getInfo() > 0:
        mediana = collection_procesada.median().set({
            'periodo': periodo_key,
            'sensor': periodo['sensor'],
            'start_date': periodo['start'],
            'end_date': periodo['end'],
            'n_images': collection_procesada.size()
        })
        
        print(f"✓ Composición mediana creada para {periodo_key}")
        return mediana
    else:
        print(f"⚠️  No hay imágenes válidas para {periodo_key}")
        return None

# Procesar cada década de Landsat
print("=== INICIANDO PROCESAMIENTO MULTITEMPORAL ===")

# Usar geometría de todas las AOI para filtrado espacial
aoi_bounds = aois.geometry().bounds()

# Procesar décadas
composiciones = {}

for periodo_key in ['1990s', '2000s', '2010s']:
    comp = procesar_landsat_decada(periodo_key, aoi_bounds)
    if comp is not None:
        composiciones[periodo_key] = comp

print(f"\n✓ Procesamiento completado")
print(f"Composiciones creadas: {list(composiciones.keys())}")

# Verificar disponibilidad de datos
if len(composiciones) >= 3:
    print("✓ Suficientes datos para análisis multitemporal (≥30 años)")
else:
    print("⚠️  Datos limitados. Considera ajustar filtros o área de estudio")

=== INICIANDO PROCESAMIENTO MULTITEMPORAL ===

=== PROCESANDO Landsat 5 TM (1990s) ===
Imágenes antes de filtros: 3
Imágenes época seca: 0
Imágenes procesadas: 0
⚠️  No hay imágenes válidas para 1990s

=== PROCESANDO Landsat 7 ETM+ (2000s) ===
Imágenes antes de filtros: 3
Imágenes época seca: 2
Imágenes procesadas: 2
✓ Composición mediana creada para 2000s

=== PROCESANDO Landsat 8 OLI (2010s) ===
Imágenes antes de filtros: 2
Imágenes época seca: 1
Imágenes procesadas: 1
✓ Composición mediana creada para 2010s

✓ Procesamiento completado
Composiciones creadas: ['2000s', '2010s']
⚠️  Datos limitados. Considera ajustar filtros o área de estudio


In [8]:
# --- OPTIMIZACIÓN: Ajustar filtros para mayor disponibilidad ---

def procesar_landsat_decada_flexible(periodo_key, aoi_geometry, cloud_max=50, incluir_toda_epoca=False):
    """
    Versión más flexible del procesamiento con filtros ajustables
    """
    periodo = PERIODOS[periodo_key]
    print(f"\n=== REPROCESANDO {periodo['nombre']} (FILTROS FLEXIBLES) ===")
    
    # Crear colección base con filtros más permisivos
    collection = ee.ImageCollection(periodo['sensor']) \
        .filterDate(periodo['start'], periodo['end']) \
        .filterBounds(aoi_geometry) \
        .filter(ee.Filter.lt('CLOUD_COVER', cloud_max))
    
    print(f"Imágenes con nubes <{cloud_max}%: {collection.size().getInfo()}")
    
    # Aplicar filtro estacional más flexible si es necesario
    if incluir_toda_epoca:
        # Incluir época seca + transición (Nov-Abr)
        meses_ampliados = [11, 12, 1, 2, 3, 4]
        collection_epoca = filtrar_epoca_seca(collection, meses_ampliados)
        print(f"Imágenes época seca ampliada (Nov-Abr): {collection_epoca.size().getInfo()}")
    else:
        collection_epoca = filtrar_epoca_seca(collection, EPOCA_SECA)
        print(f"Imágenes época seca: {collection_epoca.size().getInfo()}")
    
    # Si aún no hay suficientes imágenes, usar toda la colección
    if collection_epoca.size().getInfo() == 0:
        print("⚠️  Sin imágenes en época seca. Usando todas las imágenes disponibles...")
        collection_epoca = collection
    
    # Aplicar filtros de nubes y calcular índices (solo Landsat)
    collection_procesada = collection_epoca \
        .map(filtrar_nubes_landsat_c2) \
        .map(calcular_indices_landsat)
    
    print(f"Imágenes procesadas: {collection_procesada.size().getInfo()}")
    
    # Crear composición mediana
    if collection_procesada.size().getInfo() > 0:
        mediana = collection_procesada.median().set({
            'periodo': periodo_key,
            'sensor': periodo['sensor'],
            'start_date': periodo['start'],
            'end_date': periodo['end'],
            'n_images': collection_procesada.size(),
            'cloud_max': cloud_max,
            'epoca_ampliada': incluir_toda_epoca
        })
        
        print(f"✓ Composición mediana creada para {periodo_key}")
        return mediana
    else:
        print(f"❌ No hay imágenes válidas para {periodo_key}")
        return None

# Reprocesar décadas con filtros más flexibles
print("=== REPROCESAMIENTO CON FILTROS OPTIMIZADOS ===")

# Para 1990s: usar filtros muy permisivos debido a datos limitados
if '1990s' not in composiciones:
    comp_1990s = procesar_landsat_decada_flexible('1990s', aoi_bounds, 
                                                  cloud_max=70, incluir_toda_epoca=True)
    if comp_1990s is not None:
        composiciones['1990s'] = comp_1990s

# Verificar si necesitamos mejorar otras décadas
decadas_mejorar = []
for decada in ['2000s', '2010s']:
    if decada in composiciones:
        n_images = composiciones[decada].get('n_images').getInfo()
        if n_images < 3:  # Si hay muy pocas imágenes
            decadas_mejorar.append(decada)

if decadas_mejorar:
    print(f"Mejorando décadas con pocos datos: {decadas_mejorar}")
    for decada in decadas_mejorar:
        comp_mejorada = procesar_landsat_decada_flexible(decada, aoi_bounds,
                                                       cloud_max=40, incluir_toda_epoca=True)
        if comp_mejorada is not None:
            composiciones[decada] = comp_mejorada

print(f"\n✓ Reprocesamiento completado")
print(f"Composiciones finales: {list(composiciones.keys())}")

# Resumen de datos disponibles
print("\n=== RESUMEN DATOS MULTITEMPORALES ===")
for periodo, comp in composiciones.items():
    n_imgs = comp.get('n_images').getInfo()
    print(f"{periodo}: {n_imgs} imágenes")

anos_cubiertos = len(composiciones)
if anos_cubiertos >= 3:
    print(f"✓ Análisis multitemporal viable con {anos_cubiertos} décadas")
else:
    print(f"⚠️  Solo {anos_cubiertos} décadas disponibles. Considera área de estudio diferente.")

=== REPROCESAMIENTO CON FILTROS OPTIMIZADOS ===

=== REPROCESANDO Landsat 5 TM (1990s) (FILTROS FLEXIBLES) ===
Imágenes con nubes <70%: 36
Imágenes época seca ampliada (Nov-Abr): 14
Imágenes procesadas: 14
✓ Composición mediana creada para 1990s
Mejorando décadas con pocos datos: ['2000s', '2010s']

=== REPROCESANDO Landsat 7 ETM+ (2000s) (FILTROS FLEXIBLES) ===
Imágenes con nubes <40%: 19
Imágenes época seca ampliada (Nov-Abr): 8
Imágenes procesadas: 8
✓ Composición mediana creada para 2000s

=== REPROCESANDO Landsat 8 OLI (2010s) (FILTROS FLEXIBLES) ===
Imágenes con nubes <40%: 6
Imágenes época seca ampliada (Nov-Abr): 5
Imágenes procesadas: 5
✓ Composición mediana creada para 2010s

✓ Reprocesamiento completado
Composiciones finales: ['2000s', '2010s', '1990s']

=== RESUMEN DATOS MULTITEMPORALES ===
2000s: 8 imágenes
2010s: 5 imágenes
1990s: 14 imágenes
✓ Análisis multitemporal viable con 3 décadas


## SECCIÓN 5: Sentinel-2 para Presente (1999-2025) + Fusión de Datos

Procesamiento de Sentinel-2 y fusión con Landsat 8 para el período más reciente.

In [9]:
# --- PARTE 5: Sentinel-2 y fusión período extendido ---

def procesar_sentinel2_extendido(aoi_geometry):
    """
    Procesa Sentinel-2 para el período extendido (1999-2025)
    """
    print("\n=== PROCESANDO SENTINEL-2 (1999-2025) ===")
    
    # Colección Sentinel-2 Surface Reflectance (disponible desde 2015)
    s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate('2015-01-01', '2025-12-31') \
        .filterBounds(aoi_geometry) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_MAX))
    
    print(f"Imágenes S2 antes de filtros: {s2_collection.size().getInfo()}")
    
    # Filtrar por época seca
    s2_epoca_seca = filtrar_epoca_seca(s2_collection, EPOCA_SECA)
    print(f"Imágenes S2 época seca: {s2_epoca_seca.size().getInfo()}")
    
    # Aplicar filtros de nubes y calcular índices
    s2_procesada = s2_epoca_seca \
        .map(filtrar_nubes_sentinel2) \
        .map(calcular_indices_sentinel2)
    
    print(f"Imágenes S2 procesadas: {s2_procesada.size().getInfo()}")
    
    if s2_procesada.size().getInfo() > 0:
        s2_mediana = s2_procesada.median().set({
            'periodo': 'S2_extendido',
            'sensor': 'Sentinel-2',
            'start_date': '2015-01-01',
            'end_date': '2025-12-31',
            'n_images': s2_procesada.size()
        })
        
        print("✓ Composición S2 extendida creada")
        return s2_mediana
    else:
        print("⚠️  No hay imágenes S2 válidas")
        return None

def procesar_landsat8_extendido(aoi_geometry):
    """
    Procesa Landsat 8 para el período extendido (1999-2025)
    """
    print("\n=== PROCESANDO LANDSAT 8 EXTENDIDO (1999-2025) ===")
    
    # Landsat 8 está disponible desde 2013
    l8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('1999-01-01', '2025-12-31') \
        .filterBounds(aoi_geometry) \
        .filter(ee.Filter.lt('CLOUD_COVER', CLOUD_COVER_MAX))
    
    print(f"Imágenes L8 antes de filtros: {l8_collection.size().getInfo()}")
    
    # Filtrar por época seca
    l8_epoca_seca = filtrar_epoca_seca(l8_collection, EPOCA_SECA)
    print(f"Imágenes L8 época seca: {l8_epoca_seca.size().getInfo()}")
    
    # Aplicar filtros de nubes y calcular índices
    l8_procesada = l8_epoca_seca \
        .map(filtrar_nubes_landsat_c2) \
        .map(calcular_indices_landsat)
    
    print(f"Imágenes L8 procesadas: {l8_procesada.size().getInfo()}")
    
    if l8_procesada.size().getInfo() > 0:
        l8_mediana = l8_procesada.median().set({
            'periodo': 'L8_extendido',
            'sensor': 'Landsat-8',
            'start_date': '1999-01-01',
            'end_date': '2025-12-31',
            'n_images': l8_procesada.size()
        })
        
        print("✓ Composición L8 extendida creada")
        return l8_mediana
    else:
        print("⚠️  No hay imágenes L8 válidas")
        return None

def fusionar_landsat8_sentinel2(l8_comp, s2_comp):
    """
    Fusiona composiciones de Landsat 8 y Sentinel-2 para período extendido
    """
    print("\n=== FUSIONANDO L8 + S2 PERÍODO EXTENDIDO ===")
    
    if l8_comp is None and s2_comp is None:
        print("⚠️  Ninguna composición disponible")
        return None
    elif l8_comp is None:
        print("⚠️  Solo S2 disponible")
        return s2_comp
    elif s2_comp is None:
        print("⚠️  Solo L8 disponible")
        return l8_comp
    
    # Seleccionar bandas comunes (índices)
    l8_indices = l8_comp.select(['NDVI', 'NBR'])
    s2_indices = s2_comp.select(['NDVI', 'NBR'])
    
    # Crear máscara de datos válidos
    l8_mask = l8_indices.mask().reduce(ee.Reducer.allNonZero())
    s2_mask = s2_indices.mask().reduce(ee.Reducer.allNonZero())
    
    # Fusión: usar S2 donde esté disponible, L8 como respaldo
    fusion_ndvi = s2_indices.select('NDVI').where(s2_mask.Not(), l8_indices.select('NDVI'))
    fusion_nbr = s2_indices.select('NBR').where(s2_mask.Not(), l8_indices.select('NBR'))
    
    # RGB de mayor resolución (S2)
    fusion_rgb = s2_comp.select(['Red', 'Green', 'Blue']).where(
        s2_mask.Not(), l8_comp.select(['Red', 'Green', 'Blue'])
    )
    
    # Combinar todas las bandas
    fusion = fusion_ndvi.addBands(fusion_nbr).addBands(fusion_rgb).set({
        'periodo': 'fusion_extendido',
        'sensor': 'Landsat8+Sentinel2',
        'fusion_method': 'S2_priority',
        'start_date': '1999-01-01',
        'end_date': '2025-12-31'
    })
    
    print("✓ Fusión L8+S2 extendida completada")
    return fusion

# Ejecutar procesamiento período extendido
print("=== PROCESANDO PERÍODO EXTENDIDO (1999-2025) ===")

# Procesar Landsat 8 extendido
l8_extendido = procesar_landsat8_extendido(aoi_bounds)
if l8_extendido is not None:
    composiciones['L8_extendido'] = l8_extendido

# Procesar Sentinel-2
s2_extendido = procesar_sentinel2_extendido(aoi_bounds)
if s2_extendido is not None:
    composiciones['S2_extendido'] = s2_extendido

# Fusionar datos del período extendido
fusion_extendida = fusionar_landsat8_sentinel2(l8_extendido, s2_extendido)
if fusion_extendida is not None:
    composiciones['fusion_extendido'] = fusion_extendida

print(f"\n✓ Procesamiento período extendido completado")
print(f"Composiciones disponibles: {list(composiciones.keys())}")

=== PROCESANDO PERÍODO EXTENDIDO (1999-2025) ===

=== PROCESANDO LANDSAT 8 EXTENDIDO (1999-2025) ===
Imágenes L8 antes de filtros: 3
Imágenes L8 época seca: 2
Imágenes L8 procesadas: 2
✓ Composición L8 extendida creada

=== PROCESANDO SENTINEL-2 (1999-2025) ===
Imágenes S2 antes de filtros: 36
Imágenes S2 época seca: 13
Imágenes S2 procesadas: 13
✓ Composición S2 extendida creada

=== FUSIONANDO L8 + S2 PERÍODO EXTENDIDO ===
✓ Fusión L8+S2 extendida completada

✓ Procesamiento período extendido completado
Composiciones disponibles: ['2000s', '2010s', '1990s', 'L8_extendido', 'S2_extendido', 'fusion_extendido']


## SECCIÓN 6: Visualización Interactiva y Comparación

Creación de mapas interactivos para visualizar y comparar las composiciones multitemporales.

In [10]:
# --- PARTE 6: Mapas estáticos para informe ---

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle, FancyArrowPatch
from matplotlib import patheffects as pe
import numpy as np
import requests
from PIL import Image
import io
import math
from datetime import datetime

# Parámetros de visualización
VIS_RGB = {
    'bands': ['Red', 'Green', 'Blue'],
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

VIS_NDVI = {
    'bands': ['NDVI'],
    'min': -0.5,
    'max': 0.9,
    'palette': ['red', 'yellow', 'green']
}

VIS_NDWI = {
    'bands': ['NDWI'],
    'min': -0.8,
    'max': 0.8,
    'palette': ['brown', 'white', 'blue']
}

VIS_CAMBIO_NDVI = {
    'bands': ['dNDVI'],
    'min': -0.4,
    'max': 0.4,
    'palette': ['red', 'white', 'green']
}

def _extraer_xy_nombres(puntos_fc):
    """
    Extrae coordenadas y nombres de puntos desde un FeatureCollection
    """
    info = puntos_fc.getInfo()
    xs = [f['geometry']['coordinates'][0] for f in info['features']]
    ys = [f['geometry']['coordinates'][1] for f in info['features']]
    names = [f['properties'].get('nombre', f'P{i+1}') for i, f in enumerate(info['features'])]
    return xs, ys, names

def calcular_escala_aproximada(min_lon, max_lon, min_lat, max_lat):
    """
    Calcula escala aproximada en metros por pixel basada en bounds geográficos
    """
    # Distancia horizontal aproximada en grados
    lat_centro = (min_lat + max_lat) / 2
    
    # Conversión aproximada de grados a metros en el ecuador
    # 1 grado ≈ 111320 metros, ajustado por latitud
    metros_por_grado_lon = 111320 * math.cos(math.radians(lat_centro))
    metros_por_grado_lat = 111320
    
    # Calcular dimensiones aproximadas
    ancho_m = (max_lon - min_lon) * metros_por_grado_lon
    alto_m = (max_lat - min_lat) * metros_por_grado_lat
    
    return ancho_m, alto_m

def agregar_flecha_norte(ax, aoi_bounds):
    """
    Agrega flecha norte al mapa
    """
    min_lon, min_lat = aoi_bounds['coordinates'][0][0]
    max_lon, max_lat = aoi_bounds['coordinates'][0][2]
    
    # Posición en esquina superior derecha
    x_norte = max_lon - (max_lon - min_lon) * 0.08
    y_norte = max_lat - (max_lat - min_lat) * 0.08
    
    # Crear flecha norte
    arrow = FancyArrowPatch(
        (x_norte, y_norte - (max_lat - min_lat) * 0.04),
        (x_norte, y_norte),
        arrowstyle='->', mutation_scale=15, linewidth=2,
        color='black', zorder=10
    )
    ax.add_patch(arrow)
    
    # Etiqueta "N"
    ax.text(x_norte, y_norte + (max_lat - min_lat) * 0.02, 'N', 
           fontsize=12, weight='bold', ha='center', va='bottom',
           zorder=10)

def agregar_barra_escala(ax, aoi_bounds):
    """
    Agrega barra de escala manual al mapa
    """
    min_lon, min_lat = aoi_bounds['coordinates'][0][0]
    max_lon, max_lat = aoi_bounds['coordinates'][0][2]
    
    # Calcular escala aproximada
    ancho_m, alto_m = calcular_escala_aproximada(min_lon, max_lon, min_lat, max_lat)
    
    # Determinar longitud apropiada de barra (aprox 20% del ancho)
    longitud_deseada = ancho_m * 0.2
    
    # Redondear a valores apropiados (100m, 200m, 500m, 1km, etc.)
    if longitud_deseada < 100:
        longitud_barra = 50
    elif longitud_deseada < 200:
        longitud_barra = 100
    elif longitud_deseada < 500:
        longitud_barra = 200
    elif longitud_deseada < 1000:
        longitud_barra = 500
    elif longitud_deseada < 2000:
        longitud_barra = 1000
    else:
        longitud_barra = round(longitud_deseada / 1000) * 1000
    
    # Convertir metros a grados (aproximado)
    lat_centro = (min_lat + max_lat) / 2
    metros_por_grado = 111320 * math.cos(math.radians(lat_centro))
    longitud_grados = longitud_barra / metros_por_grado
    
    # Posición en esquina inferior derecha
    x_escala = max_lon - (max_lon - min_lon) * 0.25
    y_escala = min_lat + (max_lat - min_lat) * 0.08
    
    # Dibujar barra
    ax.plot([x_escala, x_escala + longitud_grados], [y_escala, y_escala],
           'k-', linewidth=3, zorder=10)
    ax.plot([x_escala, x_escala], [y_escala - (max_lat - min_lat) * 0.01, 
                                   y_escala + (max_lat - min_lat) * 0.01],
           'k-', linewidth=2, zorder=10)
    ax.plot([x_escala + longitud_grados, x_escala + longitud_grados], 
           [y_escala - (max_lat - min_lat) * 0.01, 
            y_escala + (max_lat - min_lat) * 0.01],
           'k-', linewidth=2, zorder=10)
    
    # Etiqueta
    if longitud_barra >= 1000:
        etiqueta = f"{longitud_barra/1000:.1f} km"
    else:
        etiqueta = f"{longitud_barra} m"
    
    ax.text(x_escala + longitud_grados/2, y_escala + (max_lat - min_lat) * 0.025,
           etiqueta, fontsize=10, weight='bold', ha='center', va='bottom',
           bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8),
           zorder=10)

def crear_leyenda_completa(tipo_viz, epoca_info, puntos_fc=None):
    """
    Crea elementos de leyenda completa según el tipo de visualización
    """
    legend_elements = []
    
    # Elementos base según tipo
    if tipo_viz == 'RGB':
        legend_elements.append(patches.Patch(color='none', 
                              label=f'Composición RGB'))
        legend_elements.append(patches.Patch(color='none', 
                              label=f'Época: {epoca_info}'))
        legend_elements.append(patches.Patch(color='none', 
                              label='Bandas: Red, Green, Blue'))
        
    elif tipo_viz == 'NDVI':
        legend_elements.append(patches.Patch(color='red', 
                              label='NDVI Bajo (<0.2)'))
        legend_elements.append(patches.Patch(color='yellow', 
                              label='NDVI Medio (0.2-0.6)'))
        legend_elements.append(patches.Patch(color='green', 
                              label='NDVI Alto (>0.6)'))
        legend_elements.append(patches.Patch(color='none', 
                              label=f'Época: {epoca_info}'))
                              
    elif tipo_viz == 'dNDVI':
        legend_elements.append(patches.Patch(color='red', 
                              label='Pérdida vegetación'))
        legend_elements.append(patches.Patch(color='white', 
                              label='Sin cambio'))
        legend_elements.append(patches.Patch(color='green', 
                              label='Ganancia vegetación'))
        legend_elements.append(patches.Patch(color='none', 
                              label=f'Periodo: {epoca_info}'))
    
    # Agregar puntos críticos si existen
    if puntos_fc is not None:
        try:
            num_puntos = puntos_fc.size().getInfo()
            if num_puntos > 0:
                legend_elements.append(patches.Patch(color='yellow', 
                                      label=f'Puntos críticos ({num_puntos})'))
        except:
            legend_elements.append(patches.Patch(color='yellow', 
                                  label='Puntos críticos'))
    
    return legend_elements

def descargar_imagen_gee(image_ee, region, vis_params=None, nombre_archivo="img"):
    """
    Descarga imagen real desde Google Earth Engine
    vis_params es opcional - se usa None para imágenes ya visualizadas
    """
    try:
        # Obtener URL del thumbnail desde GEE
        params = {
            'region': region,
            'dimensions': 1024,
            'format': 'png'
        }
        
        # Solo agregar vis_params si no es None
        if vis_params:
            params.update(vis_params)
            
        url = image_ee.getThumbURL(params)
        
        # Descargar la imagen
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        # Guardar localmente
        ruta_local = OUT_DIR / f"{nombre_archivo}.png"
        with open(ruta_local, 'wb') as f:
            f.write(response.content)
            
        print(f"  ✓ Imagen descargada: {ruta_local}")
        return str(ruta_local), url
        
    except Exception as e:
        print(f"  ✗ Error descargando imagen: {e}")
        return None, None

def crear_mapa_estatico_epoca(composicion, aoi_fc, titulo, tipo_viz='RGB', puntos_fc=None):
    """
    Crea un mapa estático para una época específica con elementos cartográficos completos
    """
    print(f"Generando mapa: {titulo}")
    
    # Limpiar el nombre del archivo
    nombre_archivo = "".join(c for c in titulo if c.isalnum() or c in (' ', '-', '_')).rstrip()
    nombre_archivo = nombre_archivo.replace(' ', '_')
    
    # Obtener bounds del AOI para el extent del mapa
    aoi_bounds = aoi_fc.geometry().bounds().getInfo()
    region = aoi_fc.geometry().bounds()
    
    # Configurar visualización según el tipo
    if tipo_viz == 'RGB' and all(b in composicion.bandNames().getInfo() for b in ['Red','Green','Blue']):
        image_viz = composicion.select(['Red','Green','Blue'])
        vis_params = VIS_RGB
        titulo_banda = "RGB Natural"
    elif tipo_viz == 'NDVI' and 'NDVI' in composicion.bandNames().getInfo():
        image_viz = composicion.select('NDVI')
        vis_params = VIS_NDVI  
        titulo_banda = "NDVI"
    elif tipo_viz == 'NDWI' and 'NDWI' in composicion.bandNames().getInfo():
        image_viz = composicion.select('NDWI')
        vis_params = VIS_NDWI
        titulo_banda = "NDWI"
    else:
        # Fallback: primera banda disponible
        image_viz = composicion.select(0)
        vis_params = {'min': 0, 'max': 0.3}
        titulo_banda = "Composición"
    
    # Descargar imagen base
    ruta_imagen, url_gee = descargar_imagen_gee(image_viz, region, vis_params, f"satelital_{nombre_archivo}")
    
    if not ruta_imagen:
        print(f"  ✗ No se pudo descargar la imagen base para: {titulo}")
        return None
        
    # Crear el mapa
    try:
        fig, ax = plt.subplots(figsize=(14, 12))
        
        # Cargar y mostrar imagen satelital
        img_satelital = Image.open(ruta_imagen)
        img_satelital = np.array(img_satelital)
        
        # Configurar extent geográfico
        min_lon, min_lat = aoi_bounds['coordinates'][0][0]
        max_lon, max_lat = aoi_bounds['coordinates'][0][2] 
        
        ax.imshow(img_satelital, extent=[min_lon, max_lon, min_lat, max_lat], aspect='auto')
        
        # OVERLAY DE PUNTOS Y ETIQUETAS
        if puntos_fc is not None and puntos_fc.size().getInfo() > 0:
            xs, ys, names = _extraer_xy_nombres(puntos_fc)
            ax.scatter(xs, ys, s=45, facecolors='yellow', edgecolors='black',
                      linewidth=1.2, zorder=5)
            for x, y, name in zip(xs, ys, names):
                ax.text(x, y, name, fontsize=9, weight='bold', color='yellow', zorder=6,
                       path_effects=[pe.withStroke(linewidth=2.5, foreground='black')])
        
        # ELEMENTOS CARTOGRÁFICOS
        agregar_flecha_norte(ax, aoi_bounds)
        agregar_barra_escala(ax, aoi_bounds)
        
        # LEYENDA COMPLETA
        legend_elements = crear_leyenda_completa(tipo_viz, titulo, puntos_fc)
        ax.legend(handles=legend_elements, loc='upper left', fontsize=10,
                 bbox_to_anchor=(0.02, 0.98), frameon=True, fancybox=True,
                 shadow=True, framealpha=0.9)
        
        # Configurar ejes y labels
        ax.set_xlabel('Longitud (°)', fontsize=12)
        ax.set_ylabel('Latitud (°)', fontsize=12)
        ax.set_title(f'{titulo}\\n{titulo_banda}', fontsize=16, weight='bold', pad=20)
        
        # Grid con coordenadas
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        ax.tick_params(axis='both', which='major', labelsize=10)
        
        # Información adicional en la parte inferior
        info_text = f"Datum: WGS84 | CRS: EPSG:4326 | Generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
        ax.text(0.5, 0.02, info_text, transform=ax.transAxes, ha='center',
               fontsize=8, style='italic', bbox=dict(boxstyle="round,pad=0.3", 
               facecolor='white', alpha=0.8))
        
        # Ajustar layout
        plt.tight_layout()
        
        # Guardar mapa final
        ruta_mapa = OUT_DIR / f"mapa_{nombre_archivo}.png"
        plt.savefig(ruta_mapa, dpi=300, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
        plt.close()
        
        print(f"  ✓ Mapa guardado: {ruta_mapa}")
        return str(ruta_mapa)
        
    except Exception as e:
        print(f"  ✗ Error creando mapa: {e}")
        plt.close()
        return None

def crear_mapa_cambios(imagen_cambio, aoi_fc, titulo, tipo_cambio='dNDVI', puntos_fc=None):
    """
    Crea mapa de cambios temporales con elementos cartográficos completos
    """
    print(f"Generando mapa de cambios: {titulo}")
    
    # Limpiar nombre archivo
    nombre_archivo = "".join(c for c in titulo if c.isalnum() or c in (' ', '-', '_')).rstrip()
    nombre_archivo = nombre_archivo.replace(' ', '_')
    
    # Obtener bounds del AOI
    aoi_bounds = aoi_fc.geometry().bounds().getInfo()
    region = aoi_fc.geometry().bounds()
    
    # Configurar visualización según tipo de cambio
    if tipo_cambio == 'dNDVI':
        vis_params = VIS_CAMBIO_NDVI
        titulo_banda = "Cambio en NDVI"
    else:
        vis_params = {'min': -0.3, 'max': 0.3, 'palette': ['red', 'white', 'blue']}
        titulo_banda = f"Cambio en {tipo_cambio}"
    
    # Descargar imagen de cambios
    ruta_imagen, url_gee = descargar_imagen_gee(imagen_cambio, region, vis_params, f"cambios_{nombre_archivo}")
    
    if not ruta_imagen:
        print(f"  ✗ No se pudo descargar la imagen de cambios para: {titulo}")
        return None
        
    try:
        fig, ax = plt.subplots(figsize=(14, 12))
        
        # Cargar imagen de cambios
        img_cambios = Image.open(ruta_imagen)
        img_cambios = np.array(img_cambios)
        
        # Configurar extent geográfico
        min_lon, min_lat = aoi_bounds['coordinates'][0][0]
        max_lon, max_lat = aoi_bounds['coordinates'][0][2]
        
        ax.imshow(img_cambios, extent=[min_lon, max_lon, min_lat, max_lat], aspect='auto')
        
        # OVERLAY DE PUNTOS Y ETIQUETAS
        if puntos_fc is not None and puntos_fc.size().getInfo() > 0:
            xs, ys, names = _extraer_xy_nombres(puntos_fc)
            ax.scatter(xs, ys, s=45, facecolors='yellow', edgecolors='black',
                      linewidth=1.2, zorder=5)
            for x, y, name in zip(xs, ys, names):
                ax.text(x, y, name, fontsize=9, weight='bold', color='yellow', zorder=6,
                       path_effects=[pe.withStroke(linewidth=2.5, foreground='black')])
        
        # ELEMENTOS CARTOGRÁFICOS
        agregar_flecha_norte(ax, aoi_bounds)
        agregar_barra_escala(ax, aoi_bounds)
        
        # LEYENDA COMPLETA
        legend_elements = crear_leyenda_completa(tipo_cambio, titulo, puntos_fc)
        ax.legend(handles=legend_elements, loc='upper left', fontsize=10,
                 bbox_to_anchor=(0.02, 0.98), frameon=True, fancybox=True,
                 shadow=True, framealpha=0.9)
        
        # Configurar ejes y labels
        ax.set_xlabel('Longitud (°)', fontsize=12)
        ax.set_ylabel('Latitud (°)', fontsize=12)
        ax.set_title(f'{titulo}\\n{titulo_banda}', fontsize=16, weight='bold', pad=20)
        
        # Grid con coordenadas
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        ax.tick_params(axis='both', which='major', labelsize=10)
        
        # Información adicional
        info_text = f"Datum: WGS84 | CRS: EPSG:4326 | Generado: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
        ax.text(0.5, 0.02, info_text, transform=ax.transAxes, ha='center',
               fontsize=8, style='italic', bbox=dict(boxstyle="round,pad=0.3", 
               facecolor='white', alpha=0.8))
        
        # Ajustar layout
        plt.tight_layout()
        
        # Guardar mapa final
        ruta_mapa = OUT_DIR / f"mapa_cambios_{nombre_archivo}.png"
        plt.savefig(ruta_mapa, dpi=300, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        plt.close()
        
        print(f"  ✓ Mapa de cambios guardado: {ruta_mapa}")
        return str(ruta_mapa)
        
    except Exception as e:
        print(f"  ✗ Error creando mapa de cambios: {e}")
        plt.close()
        return None

print("✓ Funciones de mapas estáticos definidas (con elementos cartográficos completos)")

✓ Funciones de mapas estáticos definidas (con elementos cartográficos completos)


## SECCIÓN 7: Análisis de Cambios Multitemporales

Cálculo de diferencias temporales y detección de cambios en cobertura y morfología.

In [11]:
# --- PARTE 7: Análisis de cambios temporales ---

def calcular_cambios_temporales(composiciones_dict):
    """
    Calcula diferencias temporales entre períodos para detectar cambios
    """
    print("=== CALCULANDO CAMBIOS TEMPORALES ===")
    
    cambios = {}
    
    # Definir pares de comparación temporal
    comparaciones = [
        ('1990s', '2000s', 'Cambio_1990s-2000s'),
        ('2000s', '2010s', 'Cambio_2000s-2010s'),
        ('2010s', 'L8_extendido', 'Cambio_2010s-Presente'),
        ('1990s', 'L8_extendido', 'Cambio_Total_1990s-Presente')
    ]
    
    for periodo1, periodo2, nombre_cambio in comparaciones:
        if periodo1 in composiciones_dict and periodo2 in composiciones_dict:
            print(f"Calculando: {nombre_cambio}")
            
            comp1 = composiciones_dict[periodo1]
            comp2 = composiciones_dict[periodo2]
            
            # Cambio en NDVI (pérdida/ganancia de vegetación)
            cambio_ndvi = comp2.select('NDVI').subtract(comp1.select('NDVI')).rename('dNDVI')
            
            # Cambio en NBR (detección de disturbios)
            cambio_nbr = comp2.select('NBR').subtract(comp1.select('NBR')).rename('dNBR')
            
            # Magnitud total del cambio
            magnitud_cambio = cambio_ndvi.pow(2).add(cambio_nbr.pow(2)).sqrt().rename('magnitud_cambio')
            
            # Combinar bandas de cambio
            cambio_total = cambio_ndvi.addBands([cambio_nbr, magnitud_cambio]).set({
                'tipo': 'cambio_temporal',
                'periodo_inicial': periodo1,
                'periodo_final': periodo2,
                'nombre': nombre_cambio
            })
            
            cambios[nombre_cambio] = cambio_total
            print(f"  ✓ {nombre_cambio} calculado")
    
    return cambios

def clasificar_tipos_cambio(imagen_cambio, umbral_cambio=0.1):
    """
    Clasifica píxeles según el tipo de cambio detectado
    """
    # Extraer bandas de cambio
    dndvi = imagen_cambio.select('dNDVI')
    dnbr = imagen_cambio.select('dNBR')
    magnitud = imagen_cambio.select('magnitud_cambio')
    
    # Crear máscara de cambio significativo
    cambio_significativo = magnitud.gt(umbral_cambio)
    
    # Clasificar tipos de cambio
    # 1: Ganancia de vegetación (dNDVI > 0.1)
    ganancia_veg = dndvi.gt(0.1).And(cambio_significativo)
    
    # 2: Pérdida de vegetación (dNDVI < -0.1)
    perdida_veg = dndvi.lt(-0.1).And(cambio_significativo)
    
    # 3: Disturbio/quema (dNBR < -0.1)
    disturbio = dnbr.lt(-0.1).And(cambio_significativo)
    
    # 4: Estable (magnitud < umbral)
    estable = cambio_significativo.Not()
    
    # Crear imagen de clasificación
    clasificacion = ee.Image(0) \
        .where(ganancia_veg, 1) \
        .where(perdida_veg, 2) \
        .where(disturbio, 3) \
        .where(estable, 0) \
        .rename('tipo_cambio')
    
    return clasificacion.addBands(magnitud)

# Calcular cambios temporales
cambios_temporales = calcular_cambios_temporales(composiciones)

print(f"\n✓ Análisis de cambios completado")
print(f"Cambios calculados: {list(cambios_temporales.keys())}")

# Clasificar tipos de cambio para el período total
if 'Cambio_Total_1990s-Presente' in cambios_temporales:
    cambio_total = cambios_temporales['Cambio_Total_1990s-Presente']
    clasificacion_cambios = clasificar_tipos_cambio(cambio_total)
    cambios_temporales['Clasificacion_Total'] = clasificacion_cambios
    print("✓ Clasificación de cambios creada")

=== CALCULANDO CAMBIOS TEMPORALES ===
Calculando: Cambio_1990s-2000s
  ✓ Cambio_1990s-2000s calculado
Calculando: Cambio_2000s-2010s
  ✓ Cambio_2000s-2010s calculado
Calculando: Cambio_2010s-Presente
  ✓ Cambio_2010s-Presente calculado
Calculando: Cambio_Total_1990s-Presente
  ✓ Cambio_Total_1990s-Presente calculado

✓ Análisis de cambios completado
Cambios calculados: ['Cambio_1990s-2000s', 'Cambio_2000s-2010s', 'Cambio_2010s-Presente', 'Cambio_Total_1990s-Presente']
✓ Clasificación de cambios creada


In [12]:
from pathlib import Path

def generar_serie_mapas_epocas():
    """
    Genera mapas estáticos para cada época y los mapas de cambios
    """
    print("=== GENERANDO SERIE DE MAPAS PARA INFORME ===\n")
    
    # Acceder a las variables globales
    global composiciones, aois, cambios_temporales
    
    # Verificar que las variables necesarias están definidas
    try:
        # Verificar composiciones
        if 'composiciones' not in globals() or composiciones is None:
            print("⚠️  Variable 'composiciones' no encontrada. Ejecuta primero las secciones 1-5.")
            return []
        
        # Verificar AOIs  
        if 'aois' not in globals() or aois is None:
            print("⚠️  Variable 'aois' no encontrada. Ejecuta primero las secciones 1-2.")
            return []
            
        # Verificar cambios_temporales (puede no existir aún)
        if 'cambios_temporales' not in globals():
            print("⚠️  Variable 'cambios_temporales' no encontrada. Solo se generarán mapas de épocas.")
            cambios_temp_local = {}
        else:
            cambios_temp_local = cambios_temporales
            
    except Exception as e:
        print(f"Error verificando variables: {e}")
        return []
    
    mapas_generados = []
    
    # 1. Mapas por época (RGB y NDVI)
    epocas_principales = {
        '1990s': 'Década 1990s - Landsat 5',
        '2000s': 'Década 2000s - Landsat 7', 
        '2010s': 'Década 2010s - Landsat 8',
        'fusion_extendido': 'Presente (1999-2025) - Landsat 8 + Sentinel-2'
    }
    
    print("📸 MAPAS POR ÉPOCA:\n")
    
    for epoca_key, titulo in epocas_principales.items():
        if epoca_key in composiciones:
            print(f"Procesando época: {titulo}")
            
            # Evita múltiples getInfo: trae los nombres una vez
            try:
                band_names = composiciones[epoca_key].bandNames().getInfo()
            except Exception as e:
                print(f"    ! No pude obtener bandNames(): {e}")
                band_names = []

            # Mapa RGB
            if 'Red' in band_names:
                ruta_rgb = crear_mapa_estatico_epoca(
                    composiciones[epoca_key], aois, 
                    f"{titulo} - RGB Natural", 'RGB'
                )
                if ruta_rgb:
                    mapas_generados.append(('RGB', epoca_key, ruta_rgb))
            
            # Mapa NDVI
            if 'NDVI' in band_names:
                ruta_ndvi = crear_mapa_estatico_epoca(
                    composiciones[epoca_key], aois,
                    f"{titulo} - NDVI", 'NDVI'
                )
                if ruta_ndvi:
                    mapas_generados.append(('NDVI', epoca_key, ruta_ndvi))
            
            print(f"  ✓ Mapas de {epoca_key} generados\n")
        else:
            print(f"  ⚠️  Época {epoca_key} no disponible\n")
    
    # 2. Mapas de cambios temporales (solo si existen)
    if cambios_temp_local:
        print("📊 MAPAS DE CAMBIOS TEMPORALES:\n")
        
        cambios_principales = {
            'Cambio_1990s-2000s': 'Cambios 1990s → 2000s',
            'Cambio_2000s-2010s': 'Cambios 2000s → 2010s', 
            'Cambio_2010s-Presente': 'Cambios 2010s → Presente',
            'Cambio_Total_1990s-Presente': 'Cambio Total (1990s → Presente)'
        }
        
        for cambio_key, titulo in cambios_principales.items():
            if cambio_key in cambios_temp_local:
                print(f"Procesando cambio: {titulo}")
                
                # Mapa de cambio NDVI
                ruta_cambio_ndvi = crear_mapa_cambios(
                    cambios_temp_local[cambio_key], aois,
                    f"{titulo} - Cambio NDVI", 'dNDVI'
                )
                if ruta_cambio_ndvi:
                    mapas_generados.append(('Cambio_NDVI', cambio_key, ruta_cambio_ndvi))
                
                # Mapa de magnitud de cambio
                ruta_magnitud = crear_mapa_cambios(
                    cambios_temp_local[cambio_key], aois,
                    f"{titulo} - Magnitud de Cambio", 'magnitud'
                )
                if ruta_magnitud:
                    mapas_generados.append(('Magnitud', cambio_key, ruta_magnitud))
                
                print(f"  ✓ Mapas de cambio {cambio_key} generados\n")
            else:
                print(f"  ⚠️  Cambio {cambio_key} no disponible\n")
    else:
        print("📊 MAPAS DE CAMBIOS TEMPORALES: No disponibles (ejecuta primero Sección 7)\n")
    
    return mapas_generados

# --- Ejecutar con mejor manejo de variables ---
try:
    # Verificar variables directamente antes de ejecutar
    variables_necesarias = ['composiciones', 'aois']
    variables_faltantes = []
    
    for var in variables_necesarias:
        if var not in globals() or globals()[var] is None:
            variables_faltantes.append(var)
    
    if variables_faltantes:
        print("⚠️  Variables faltantes para generar mapas:")
        for var in variables_faltantes:
            print(f"   - {var}")
        print("\n📋 Ejecuta primero las secciones correspondientes:")
        print("   - Secciones 1-5: para 'composiciones' y 'aois'")
        print("   - Sección 7: para 'cambios_temporales' (opcional)")
    else:
        # Variables disponibles, ejecutar función
        mapas_serie = generar_serie_mapas_epocas()
        
        print("\n=== RESUMEN MAPAS GENERADOS ===\n")
        print(f"Total de mapas creados: {len(mapas_serie)}\n")
        
        # Agrupar por tipo
        tipos_mapas = {}
        for tipo, epoca, ruta in mapas_serie:
            tipos_mapas.setdefault(tipo, []).append((epoca, ruta))
        
        for tipo, mapas in tipos_mapas.items():
            print(f"📁 {tipo}: {len(mapas)} mapas")
            for epoca, ruta in mapas:
                # Soporta str o Path
                nombre = ruta.name if hasattr(ruta, "name") else Path(ruta).name
                print(f"   - {epoca}: {nombre}")
            print()
        
        if mapas_serie:
            print(f"🗂️  Todos los mapas guardados en: {OUT_DIR}")
            print("📋 Listos para incluir en informe técnico")
            
            # Mostrar ubicaciones específicas de archivos
            print(f"\n📍 UBICACIONES DE ARCHIVOS:")
            print(f"   - Imágenes satelitales: {OUT_DIR}/satelital_*.png")
            print(f"   - Mapas finales: {OUT_DIR}/mapa_final_*.png") 
            print(f"   - Mapas de cambios: {OUT_DIR}/mapa_cambio_final_*.png")
        
except Exception as e:
    print(f"❌ Error ejecutando generación de mapas: {e}")
    print("💡 Sugerencia: Ejecuta las secciones 1-5 del notebook primero.")

=== GENERANDO SERIE DE MAPAS PARA INFORME ===

📸 MAPAS POR ÉPOCA:

Procesando época: Década 1990s - Landsat 5
Generando mapa: Década 1990s - Landsat 5 - RGB Natural
  ✓ Imagen descargada: salidas_aoi/satelital_Década_1990s_-_Landsat_5_-_RGB_Natural.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_1990s_-_Landsat_5_-_RGB_Natural.png
Generando mapa: Década 1990s - Landsat 5 - NDVI
  ✓ Imagen descargada: salidas_aoi/satelital_Década_1990s_-_Landsat_5_-_NDVI.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_1990s_-_Landsat_5_-_NDVI.png
  ✓ Mapas de 1990s generados

Procesando época: Década 2000s - Landsat 7
Generando mapa: Década 2000s - Landsat 7 - RGB Natural
  ✓ Imagen descargada: salidas_aoi/satelital_Década_2000s_-_Landsat_7_-_RGB_Natural.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_2000s_-_Landsat_7_-_RGB_Natural.png
Generando mapa: Década 2000s - Landsat 7 - NDVI
  ✓ Imagen descargada: salidas_aoi/satelital_Década_2000s_-_Landsat_7_-_NDVI.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_

In [13]:
def generar_serie_mapas_epocas():
    """
    Genera mapas estáticos para cada época y los mapas de cambios
    Incluye overlay de puntos críticos en todos los mapas
    """
    print("=== GENERANDO SERIE DE MAPAS PARA INFORME ===\n")
    
    # Acceder a las variables globales
    global composiciones, aois, cambios_temporales, puntos_fc
    
    # Verificar que las variables necesarias están definidas
    try:
        # Verificar composiciones
        if 'composiciones' not in globals() or composiciones is None:
            print("⚠️  Variable 'composiciones' no encontrada. Ejecuta primero las secciones 1-5.")
            return []
        
        # Verificar AOIs  
        if 'aois' not in globals() or aois is None:
            print("⚠️  Variable 'aois' no encontrada. Ejecuta primero las secciones 1-2.")
            return []
            
        # Verificar puntos_fc
        if 'puntos_fc' not in globals() or puntos_fc is None:
            print("⚠️  Variable 'puntos_fc' no encontrada. Se generarán mapas sin overlay de puntos.")
            puntos_fc = None
            
        # Verificar cambios_temporales (puede no existir aún)
        if 'cambios_temporales' not in globals():
            print("⚠️  Variable 'cambios_temporales' no encontrada. Solo se generarán mapas de épocas.")
            cambios_temp_local = {}
        else:
            cambios_temp_local = cambios_temporales
            
    except Exception as e:
        print(f"Error verificando variables: {e}")
        return []
    
    mapas_generados = []
    
    # 1. Mapas por época (RGB y NDVI)
    epocas_principales = {
        '1990s': 'Década 1990s - Landsat 5',
        '2000s': 'Década 2000s - Landsat 7', 
        '2010s': 'Década 2010s - Landsat 8',
        'fusion_extendido': 'Presente (1999-2025) - Landsat 8 + Sentinel-2'
    }
    
    print("📸 MAPAS POR ÉPOCA:\n")
    
    for epoca_key, titulo in epocas_principales.items():
        if epoca_key in composiciones:
            print(f"Procesando época: {titulo}")
            
            # Evita múltiples getInfo: trae los nombres una vez
            try:
                band_names = composiciones[epoca_key].bandNames().getInfo()
            except Exception as e:
                print(f"    ! No pude obtener bandNames(): {e}")
                band_names = []

            # Mapa RGB
            if 'Red' in band_names:
                ruta_rgb = crear_mapa_estatico_epoca(
                    composiciones[epoca_key], aois, 
                    f"{titulo} - RGB Natural", 'RGB',
                    puntos_fc=puntos_fc
                )
                if ruta_rgb:
                    mapas_generados.append(('RGB', epoca_key, ruta_rgb))
            
            # Mapa NDVI
            if 'NDVI' in band_names:
                ruta_ndvi = crear_mapa_estatico_epoca(
                    composiciones[epoca_key], aois,
                    f"{titulo} - NDVI", 'NDVI',
                    puntos_fc=puntos_fc
                )
                if ruta_ndvi:
                    mapas_generados.append(('NDVI', epoca_key, ruta_ndvi))
            
            print(f"  ✓ Mapas de {epoca_key} generados\n")
        else:
            print(f"  ⚠️  Época {epoca_key} no disponible\n")
    
    # 2. Mapas de cambios temporales (solo si existen)
    if cambios_temp_local:
        print("📊 MAPAS DE CAMBIOS TEMPORALES:\n")
        
        cambios_principales = {
            'Cambio_1990s-2000s': 'Cambios 1990s → 2000s',
            'Cambio_2000s-2010s': 'Cambios 2000s → 2010s', 
            'Cambio_2010s-Presente': 'Cambios 2010s → Presente',
            'Cambio_Total_1990s-Presente': 'Cambio Total (1990s → Presente)'
        }
        
        for cambio_key, titulo in cambios_principales.items():
            if cambio_key in cambios_temp_local:
                print(f"Procesando cambio: {titulo}")
                
                # Mapa de cambio NDVI
                ruta_cambio_ndvi = crear_mapa_cambios(
                    cambios_temp_local[cambio_key], aois,
                    f"{titulo} - Cambio NDVI", 'dNDVI',
                    puntos_fc=puntos_fc
                )
                if ruta_cambio_ndvi:
                    mapas_generados.append(('Cambio_NDVI', cambio_key, ruta_cambio_ndvi))
                
                # Mapa de magnitud de cambio
                ruta_magnitud = crear_mapa_cambios(
                    cambios_temp_local[cambio_key], aois,
                    f"{titulo} - Magnitud de Cambio", 'magnitud',
                    puntos_fc=puntos_fc
                )
                if ruta_magnitud:
                    mapas_generados.append(('Magnitud', cambio_key, ruta_magnitud))
                
                print(f"  ✓ Mapas de cambio {cambio_key} generados\n")
            else:
                print(f"  ⚠️  Cambio {cambio_key} no disponible\n")
    else:
        print("📊 MAPAS DE CAMBIOS TEMPORALES: No disponibles (ejecuta primero Sección 7)\n")
    
    return mapas_generados

# --- Ejecutar con mejor manejo de variables ---
try:
    # Verificar variables directamente antes de ejecutar
    variables_necesarias = ['composiciones', 'aois']
    variables_faltantes = []
    
    for var in variables_necesarias:
        if var not in globals() or globals()[var] is None:
            variables_faltantes.append(var)
    
    if variables_faltantes:
        print("⚠️  Variables faltantes para generar mapas:")
        for var in variables_faltantes:
            print(f"   - {var}")
        print("\n📋 Ejecuta primero las secciones correspondientes:")
        print("   - Secciones 1-5: para 'composiciones' y 'aois'")
        print("   - Sección 7: para 'cambios_temporales' (opcional)")
    else:
        # Variables disponibles, ejecutar función
        mapas_serie = generar_serie_mapas_epocas()
        
        print("\n=== RESUMEN MAPAS GENERADOS ===\n")
        print(f"Total de mapas creados: {len(mapas_serie)}\n")
        
        # Agrupar por tipo
        tipos_mapas = {}
        for tipo, epoca, ruta in mapas_serie:
            tipos_mapas.setdefault(tipo, []).append((epoca, ruta))
        
        for tipo, mapas in tipos_mapas.items():
            print(f"📁 {tipo}: {len(mapas)} mapas")
            for epoca, ruta in mapas:
                # Soporta str o Path
                nombre = ruta.name if hasattr(ruta, "name") else Path(ruta).name
                print(f"   - {epoca}: {nombre}")
            print()
        
        if mapas_serie:
            print(f"🗂️  Todos los mapas guardados en: {OUT_DIR}")
            print("📋 Listos para incluir en informe técnico")
            
            # Mostrar ubicaciones específicas de archivos
            print(f"\n📍 UBICACIONES DE ARCHIVOS:")
            print(f"   - Imágenes satelitales: {OUT_DIR}/satelital_*.png")
            print(f"   - Mapas finales: {OUT_DIR}/mapa_*.png") 
            print(f"   - Mapas de cambios: {OUT_DIR}/mapa_cambios_*.png")
        
except Exception as e:
    print(f"❌ Error ejecutando generación de mapas: {e}")
    print("💡 Sugerencia: Ejecuta las secciones 1-5 del notebook primero.")

=== GENERANDO SERIE DE MAPAS PARA INFORME ===

📸 MAPAS POR ÉPOCA:

Procesando época: Década 1990s - Landsat 5
Generando mapa: Década 1990s - Landsat 5 - RGB Natural
  ✓ Imagen descargada: salidas_aoi/satelital_Década_1990s_-_Landsat_5_-_RGB_Natural.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_1990s_-_Landsat_5_-_RGB_Natural.png
Generando mapa: Década 1990s - Landsat 5 - NDVI
  ✓ Imagen descargada: salidas_aoi/satelital_Década_1990s_-_Landsat_5_-_NDVI.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_1990s_-_Landsat_5_-_NDVI.png
  ✓ Mapas de 1990s generados

Procesando época: Década 2000s - Landsat 7
Generando mapa: Década 2000s - Landsat 7 - RGB Natural
  ✓ Imagen descargada: salidas_aoi/satelital_Década_2000s_-_Landsat_7_-_RGB_Natural.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_2000s_-_Landsat_7_-_RGB_Natural.png
Generando mapa: Década 2000s - Landsat 7 - NDVI
  ✓ Imagen descargada: salidas_aoi/satelital_Década_2000s_-_Landsat_7_-_NDVI.png
  ✓ Mapa guardado: salidas_aoi/mapa_Década_

## SECCIÓN 8: Tabla de Cambios Observados por Tramo

Extracción de estadísticas y generación de tabla de cambios para cada punto crítico.

In [14]:
# --- PARTE 8: Tabla de cambios por tramo ---

def extraer_estadisticas_por_aoi(composiciones_dict, cambios_dict, aoi_fc):
    """
    Extrae estadísticas completas de cada AOI (NDVI, NBR, cambios) para generar tabla de cambios
    """
    print("=== EXTRAYENDO ESTADÍSTICAS COMPLETAS POR AOI ===")
    
    # Convertir FeatureCollection a lista para procesamiento
    aois_info = aoi_fc.getInfo()
    resultados = []
    
    for i, feature in enumerate(aois_info['features']):
        aoi_geom = ee.Geometry(feature['geometry'])
        aoi_nombre = feature['properties'].get('nombre', f'AOI_{i+1}')
        
        print(f"Procesando: {aoi_nombre}")
        
        # Estadísticas por período (NDVI y NBR)
        stats_periodo = {}
        
        # Extraer NDVI y NBR promedio por período
        for periodo, comp in composiciones_dict.items():
            try:
                bandas = comp.bandNames().getInfo()
                
                # NDVI por período
                if 'NDVI' in bandas:
                    ndvi_stats = comp.select('NDVI').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,  # Usar escala estandarizada
                        maxPixels=1e9
                    ).getInfo()
                    stats_periodo[f'NDVI_{periodo}'] = ndvi_stats.get('NDVI', None)
                
                # NBR por período
                if 'NBR' in bandas:
                    nbr_stats = comp.select('NBR').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,  # Usar escala estandarizada
                        maxPixels=1e9
                    ).getInfo()
                    stats_periodo[f'NBR_{periodo}'] = nbr_stats.get('NBR', None)
                    
            except Exception as e:
                print(f"    Error procesando {periodo}: {e}")
                stats_periodo[f'NDVI_{periodo}'] = None
                stats_periodo[f'NBR_{periodo}'] = None
        
        # Estadísticas de cambios (dNDVI, dNBR, magnitud)
        stats_cambios = {}
        for nombre_cambio, cambio in cambios_dict.items():
            if nombre_cambio != 'Clasificacion_Total':
                try:
                    # Cambio promedio en NDVI
                    dndvi_stats = cambio.select('dNDVI').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,
                        maxPixels=1e9
                    ).getInfo()
                    
                    # Cambio promedio en NBR
                    dnbr_stats = cambio.select('dNBR').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,
                        maxPixels=1e9
                    ).getInfo()
                    
                    # Magnitud promedio de cambio
                    mag_stats = cambio.select('magnitud_cambio').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,
                        maxPixels=1e9
                    ).getInfo()
                    
                    stats_cambios[f'dNDVI_{nombre_cambio}'] = dndvi_stats.get('dNDVI', None)
                    stats_cambios[f'dNBR_{nombre_cambio}'] = dnbr_stats.get('dNBR', None)
                    stats_cambios[f'Magnitud_{nombre_cambio}'] = mag_stats.get('magnitud_cambio', None)
                    
                except Exception as e:
                    print(f"    Error procesando cambio {nombre_cambio}: {e}")
                    stats_cambios[f'dNDVI_{nombre_cambio}'] = None
                    stats_cambios[f'dNBR_{nombre_cambio}'] = None
                    stats_cambios[f'Magnitud_{nombre_cambio}'] = None
        
        # Clasificación de cambios (porcentajes por tipo)
        if 'Clasificacion_Total' in cambios_dict:
            try:
                clasificacion = cambios_dict['Clasificacion_Total'].select('tipo_cambio')
                
                # Contar píxeles por tipo de cambio
                area_total = aoi_geom.area().getInfo()
                
                for tipo in [0, 1, 2, 3]:  # Estable, Ganancia, Pérdida, Disturbio
                    mascara_tipo = clasificacion.eq(tipo)
                    area_tipo = mascara_tipo.multiply(ee.Image.pixelArea()).reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=aoi_geom,
                        scale=ESCALA_COMPARABILIDAD,
                        maxPixels=1e9
                    ).getInfo()
                    
                    porcentaje = (area_tipo.get('tipo_cambio', 0) / area_total) * 100
                    
                    tipos_nombres = ['Estable', 'Ganancia_Veg', 'Perdida_Veg', 'Disturbio']
                    stats_cambios[f'Pct_{tipos_nombres[tipo]}'] = porcentaje
            except Exception as e:
                print(f"    Error procesando clasificación: {e}")
                for tipo in ['Estable', 'Ganancia_Veg', 'Perdida_Veg', 'Disturbio']:
                    stats_cambios[f'Pct_{tipo}'] = 0
        
        # Combinar todas las estadísticas
        try:
            area_ha = aoi_geom.area().getInfo() / 10000  # Convertir a hectáreas
        except:
            area_ha = 0
            
        resultado = {
            'AOI': aoi_nombre,
            'Area_ha': area_ha,
            **stats_periodo,
            **stats_cambios
        }
        
        resultados.append(resultado)
        print(f"  ✓ {aoi_nombre} procesado (NDVI + NBR + cambios)")
    
    return resultados

def interpretar_cambios_completos(stats_list):
    """
    Interpreta los cambios usando NDVI y NBR para observaciones más completas
    """
    print("\n=== INTERPRETANDO CAMBIOS COMPLETOS (NDVI + NBR) ===\n")
    
    interpretaciones = []
    
    for stats in stats_list:
        aoi_nombre = stats['AOI']
        print(f"\n--- {aoi_nombre} ---")
        
        observaciones = []
        
        # Interpretar cambio total en NDVI
        if 'dNDVI_Cambio_Total_1990s-Presente' in stats:
            cambio_ndvi = stats['dNDVI_Cambio_Total_1990s-Presente']
            if cambio_ndvi is not None:
                if cambio_ndvi > 0.1:
                    observaciones.append(f"Aumento significativo de vegetación (+{cambio_ndvi:.3f} NDVI)")
                    print(f"  ✓ Ganancia de vegetación: +{cambio_ndvi:.3f}")
                elif cambio_ndvi < -0.1:
                    observaciones.append(f"Pérdida significativa de vegetación ({cambio_ndvi:.3f} NDVI)")
                    print(f"  ⚠️  Pérdida de vegetación: {cambio_ndvi:.3f}")
                else:
                    observaciones.append("Vegetación relativamente estable")
                    print(f"  ➖ Vegetación estable: {cambio_ndvi:.3f}")
        
        # Interpretar cambio total en NBR (disturbios)
        if 'dNBR_Cambio_Total_1990s-Presente' in stats:
            cambio_nbr = stats['dNBR_Cambio_Total_1990s-Presente']
            if cambio_nbr is not None:
                if cambio_nbr < -0.1:
                    observaciones.append(f"Evidencia de disturbio/quema (dNBR: {cambio_nbr:.3f})")
                    print(f"  🔥 Disturbio detectado: {cambio_nbr:.3f}")
                elif cambio_nbr > 0.1:
                    observaciones.append(f"Recuperación post-disturbio (dNBR: +{cambio_nbr:.3f})")
                    print(f"  🌱 Recuperación: +{cambio_nbr:.3f}")
        
        # Interpretar porcentajes de cambio
        if 'Pct_Perdida_Veg' in stats and stats['Pct_Perdida_Veg'] > 20:
            observaciones.append(f"Área significativa con pérdida de cobertura ({stats['Pct_Perdida_Veg']:.1f}%)")
            print(f"  🔴 Pérdida de cobertura: {stats['Pct_Perdida_Veg']:.1f}%")
        
        if 'Pct_Disturbio' in stats and stats['Pct_Disturbio'] > 10:
            observaciones.append(f"Evidencia de disturbios/cicatrices ({stats['Pct_Disturbio']:.1f}%)")
            print(f"  🟠 Disturbios detectados: {stats['Pct_Disturbio']:.1f}%")
        
        if 'Pct_Ganancia_Veg' in stats and stats['Pct_Ganancia_Veg'] > 20:
            observaciones.append(f"Recuperación de vegetación en área considerable ({stats['Pct_Ganancia_Veg']:.1f}%)")
            print(f"  🟢 Recuperación: {stats['Pct_Ganancia_Veg']:.1f}%")
        
        # Magnitud de cambio
        if 'Magnitud_Cambio_Total_1990s-Presente' in stats:
            magnitud = stats['Magnitud_Cambio_Total_1990s-Presente']
            if magnitud is not None and magnitud > 0.2:
                observaciones.append(f"Cambios de alta magnitud detectados (Magnitud: {magnitud:.3f})")
                print(f"  📊 Magnitud alta: {magnitud:.3f}")
        
        # Si no hay observaciones significativas
        if not observaciones:
            observaciones.append("Sin cambios significativos detectados en el período analizado")
            print(f"  ➡️  Sin cambios significativos")
        
        interpretaciones.append({
            'AOI': aoi_nombre,
            'Observaciones': '; '.join(observaciones)
        })
    
    return interpretaciones

# Extraer estadísticas completas (incluyendo NBR)
print("Iniciando extracción de estadísticas completas...")
estadisticas_aoi = extraer_estadisticas_por_aoi(composiciones, cambios_temporales, aois)

# Interpretar cambios completos
interpretaciones = interpretar_cambios_completos(estadisticas_aoi)

print(f"\n✓ Análisis completo para {len(estadisticas_aoi)} AOI")
print("✓ Métricas incluidas: NDVI, NBR, dNDVI, dNBR, magnitud, clasificación")

Iniciando extracción de estadísticas completas...
=== EXTRAYENDO ESTADÍSTICAS COMPLETAS POR AOI ===
Procesando: Punto 24 (k12+100)
  ✓ Punto 24 (k12+100) procesado (NDVI + NBR + cambios)
Procesando: Punto 23 (k12+040)
  ✓ Punto 23 (k12+040) procesado (NDVI + NBR + cambios)
Procesando: Punto 22 (k11+950)
  ✓ Punto 22 (k11+950) procesado (NDVI + NBR + cambios)
Procesando: Punto 21  (k11+710)
  ✓ Punto 21  (k11+710) procesado (NDVI + NBR + cambios)
Procesando: Punto 20 (k11+600)
  ✓ Punto 20 (k11+600) procesado (NDVI + NBR + cambios)
Procesando: Punto 18 (k11+330)
  ✓ Punto 18 (k11+330) procesado (NDVI + NBR + cambios)
Procesando: Punto 17 (k11+150)
  ✓ Punto 17 (k11+150) procesado (NDVI + NBR + cambios)
Procesando: Punto 16 (k10+850)
  ✓ Punto 16 (k10+850) procesado (NDVI + NBR + cambios)
Procesando: Punto 15 (k10+730)
  ✓ Punto 15 (k10+730) procesado (NDVI + NBR + cambios)
Procesando: Punto 14 (k10+490)
  ✓ Punto 14 (k10+490) procesado (NDVI + NBR + cambios)
Procesando: Punto 13 (k10+34

In [15]:
# --- Crear tabla final de resultados ---

def crear_tabla_final(estadisticas, interpretaciones):
    """
    Crea tabla final con resultados del análisis multitemporal
    """
    print("\n=== GENERANDO TABLA FINAL ===\n")
    
    # Crear DataFrame con pandas
    df_stats = pd.DataFrame(estadisticas)
    df_interp = pd.DataFrame(interpretaciones)
    
    # Combinar estadísticas e interpretaciones
    df_final = pd.merge(df_stats, df_interp, on='AOI')
    
    # Redondear valores numéricos
    columnas_numericas = df_final.select_dtypes(include=['float64', 'int64']).columns
    df_final[columnas_numericas] = df_final[columnas_numericas].round(4)
    
    # Reordenar columnas para mejor presentación
    columnas_orden = ['AOI', 'Area_ha', 'Observaciones']
    
    # Agregar columnas de NDVI por período
    ndvi_cols = [col for col in df_final.columns if col.startswith('NDVI_')]
    columnas_orden.extend(sorted(ndvi_cols))
    
    # Agregar columnas de cambios
    cambio_cols = [col for col in df_final.columns if 'Cambio_Total' in col]
    columnas_orden.extend(sorted(cambio_cols))
    
    # Agregar columnas de porcentajes
    pct_cols = [col for col in df_final.columns if col.startswith('Pct_')]
    columnas_orden.extend(sorted(pct_cols))
    
    # Agregar columnas restantes
    otras_cols = [col for col in df_final.columns if col not in columnas_orden]
    columnas_orden.extend(otras_cols)
    
    # Filtrar columnas que existen
    columnas_finales = [col for col in columnas_orden if col in df_final.columns]
    df_ordenado = df_final[columnas_finales]
    
    return df_ordenado

def generar_resumen_ejecutivo(df_tabla):
    """
    Genera resumen ejecutivo del análisis
    """
    print("=== RESUMEN EJECUTIVO ===")
    print()
    
    total_aois = len(df_tabla)
    area_total = df_tabla['Area_ha'].sum()
    
    print(f"📊 **ANÁLISIS MULTITEMPORAL PUNTOS CRÍTICOS REGION_A**")
    print(f"Período: 1990s - Presente (≥30 años)")
    print(f"Total AOI analizadas: {total_aois}")
    print(f"Área total analizada: {area_total:.1f} hectáreas")
    print()
    
    # Contar tipos de cambios predominantes
    cambios_significativos = 0
    perdida_vegetacion = 0
    ganancia_vegetacion = 0
    disturbios = 0
    
    for _, fila in df_tabla.iterrows():
        observacion = fila['Observaciones'].lower()
        
        if 'pérdida significativa' in observacion:
            perdida_vegetacion += 1
            cambios_significativos += 1
        elif 'aumento significativo' in observacion:
            ganancia_vegetacion += 1
            cambios_significativos += 1
        
        if 'disturbio' in observacion:
            disturbios += 1
    
    print(f"🔍 **HALLAZGOS PRINCIPALES:**")
    print(f"- AOI con cambios significativos: {cambios_significativos}/{total_aois} ({(cambios_significativos/total_aois)*100:.1f}%)")
    
    if perdida_vegetacion > 0:
        print(f"- AOI con pérdida de vegetación: {perdida_vegetacion} ({(perdida_vegetacion/total_aois)*100:.1f}%)")
    
    if ganancia_vegetacion > 0:
        print(f"- AOI con ganancia de vegetación: {ganancia_vegetacion} ({(ganancia_vegetacion/total_aois)*100:.1f}%)")
    
    if disturbios > 0:
        print(f"- AOI con evidencia de disturbios: {disturbios} ({(disturbios/total_aois)*100:.1f}%)")
    
    print()
    print(f"📋 **RECOMENDACIONES:**")
    if cambios_significativos > total_aois * 0.5:
        print(f"- Se detectan cambios significativos en la mayoría de puntos críticos")
        print(f"- Priorizar monitoreo continuo y medidas de estabilización")
    
    if perdida_vegetacion > 0:
        print(f"- Implementar medidas de revegetación en áreas con pérdida de cobertura")
    
    if disturbios > 0:
        print(f"- Evaluar causas de disturbios y implementar medidas preventivas")
    
    print(f"- Continuar monitoreo con imágenes de alta resolución temporal")
    
    print()
    print(f"✅ Análisis multitemporal completado exitosamente")
    print(f"📅 Período de análisis: ≥30 años (defendible)")
    print(f"🛰️  Sensores utilizados: Landsat 5, 7, 8 + Sentinel-2")

# Crear tabla final
tabla_final = crear_tabla_final(estadisticas_aoi, interpretaciones)

# Mostrar tabla
print("TABLA DE CAMBIOS MULTITEMPORALES:")
print("=" * 80)
print(tabla_final.to_string(index=False))
print("=" * 80)

# Generar resumen ejecutivo
generar_resumen_ejecutivo(tabla_final)

#Guardar tabla como CSV
ruta_salida = OUT_DIR / "tabla_cambios_multitemporales.csv"
tabla_final.to_csv(ruta_salida, index=False, encoding='utf-8')
print(f"\n💾 Tabla guardada en: {ruta_salida}")


=== GENERANDO TABLA FINAL ===

TABLA DE CAMBIOS MULTITEMPORALES:
                AOI  Area_ha                                                                                                                                                                                                 Observaciones  NDVI_1990s  NDVI_2000s  NDVI_2010s  NDVI_L8_extendido  NDVI_S2_extendido  NDVI_fusion_extendido  Magnitud_Cambio_Total_1990s-Presente  dNBR_Cambio_Total_1990s-Presente  dNDVI_Cambio_Total_1990s-Presente  Pct_Disturbio  Pct_Estable  Pct_Ganancia_Veg  Pct_Perdida_Veg  NBR_2000s  NBR_2010s  NBR_1990s  NBR_L8_extendido  NBR_S2_extendido  NBR_fusion_extendido  dNDVI_Cambio_1990s-2000s  dNBR_Cambio_1990s-2000s  Magnitud_Cambio_1990s-2000s  dNDVI_Cambio_2000s-2010s  dNBR_Cambio_2000s-2010s  Magnitud_Cambio_2000s-2010s  dNDVI_Cambio_2010s-Presente  dNBR_Cambio_2010s-Presente  Magnitud_Cambio_2010s-Presente
 Punto 24 (k12+100)  27.9365                                                                

GRAFICAS


In [16]:
tabla_final.AOI

0      Punto 24 (k12+100)
1      Punto 23 (k12+040)
2      Punto 22 (k11+950)
3     Punto 21  (k11+710)
4      Punto 20 (k11+600)
5      Punto 18 (k11+330)
6      Punto 17 (k11+150)
7      Punto 16 (k10+850)
8      Punto 15 (k10+730)
9      Punto 14 (k10+490)
10     Punto 13 (k10+340)
11      Punto 11 (k8+900)
12      Punto 12 (k9+100)
13      Punto 10 (k8+440)
14        Punto 9 (7+170)
15       Punto 8 (k6+750)
16       Punto 7 (K5+730)
17       Punto 6 (K4+880)
18      Punto 5 (K3 +115)
19       Punto 3 (K2+550)
20       Punto 4 (K1+500)
21       Punto 2 (K2+330)
22       Punto 1 (K0+890)
23     Punto 19 (k11+450)
Name: AOI, dtype: object

#INFORME

In [17]:
# ===============================
# PARTE 9: GRÁFICAS ANALÍTICAS
# ===============================
import os, re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

GRAF_DIR = OUT_DIR / "graficas"
GRAF_DIR.mkdir(parents=True, exist_ok=True)

sns.set_theme(style="whitegrid", context="talk")

# -----------------------------------------------------------------------------
# (A) Reemplazo: estadísticas por AOI -> añade NBR y dNBR
# -----------------------------------------------------------------------------
def extraer_estadisticas_por_aoi(composiciones_dict, cambios_dict, aoi_fc):
    """
    Versión extendida: además de NDVI agrega NBR por periodo y dNBR por cambio.
    """
    print("=== EXTRAYENDO ESTADÍSTICAS POR AOI (NDVI+NBR) ===")
    aois_info = aoi_fc.getInfo()
    resultados = []

    for i, feature in enumerate(aois_info['features']):
        aoi_geom = ee.Geometry(feature['geometry'])
        aoi_nombre = feature['properties'].get('nombre', f'AOI_{i+1}')
        print(f"Procesando: {aoi_nombre}")

        stats_periodo = {}

        # ---- Promedios por período (NDVI y NBR)
        for periodo, comp in composiciones_dict.items():
            try:
                bandas = comp.bandNames().getInfo()
                if 'NDVI' in bandas:
                    ndvi_stats = comp.select('NDVI').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom, scale=30, maxPixels=1e9
                    ).getInfo()
                    stats_periodo[f'NDVI_{periodo}'] = ndvi_stats.get('NDVI', None)
                if 'NBR' in bandas:
                    nbr_stats = comp.select('NBR').reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=aoi_geom, scale=30, maxPixels=1e9
                    ).getInfo()
                    stats_periodo[f'NBR_{periodo}'] = nbr_stats.get('NBR', None)
            except Exception as e:
                print(f"    Error periodo {periodo}: {e}")
                stats_periodo.setdefault(f'NDVI_{periodo}', None)
                stats_periodo.setdefault(f'NBR_{periodo}', None)

        # ---- Cambios (dNDVI, dNBR y magnitud)
        stats_cambios = {}
        for nombre_cambio, cambio in cambios_dict.items():
            if nombre_cambio == 'Clasificacion_Total':
                continue
            try:
                dndvi_stats = cambio.select('dNDVI').reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=aoi_geom, scale=30, maxPixels=1e9
                ).getInfo()
                dnbr_stats = cambio.select('dNBR').reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=aoi_geom, scale=30, maxPixels=1e9
                ).getInfo()
                mag_stats = cambio.select('magnitud_cambio').reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=aoi_geom, scale=30, maxPixels=1e9
                ).getInfo()

                stats_cambios[f'dNDVI_{nombre_cambio}'] = dndvi_stats.get('dNDVI', None)
                stats_cambios[f'dNBR_{nombre_cambio}']  = dnbr_stats.get('dNBR', None)
                stats_cambios[f'Magnitud_{nombre_cambio}'] = mag_stats.get('magnitud_cambio', None)
            except Exception as e:
                print(f"    Error cambio {nombre_cambio}: {e}")
                stats_cambios[f'dNDVI_{nombre_cambio}'] = None
                stats_cambios[f'dNBR_{nombre_cambio}']  = None
                stats_cambios[f'Magnitud_{nombre_cambio}'] = None

        # ---- Clasificación (porcentajes)
        if 'Clasificacion_Total' in cambios_dict:
            try:
                clasificacion = cambios_dict['Clasificacion_Total'].select('tipo_cambio')
                area_total = aoi_geom.area().getInfo()
                for tipo, etiqueta in zip([0,1,2,3], ['Estable','Ganancia_Veg','Perdida_Veg','Disturbio']):
                    mascara_tipo = clasificacion.eq(tipo)
                    area_tipo = mascara_tipo.multiply(ee.Image.pixelArea()).reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=aoi_geom, scale=30, maxPixels=1e9
                    ).getInfo()
                    porcentaje = (area_tipo.get('tipo_cambio', 0) / area_total) * 100
                    stats_cambios[f'Pct_{etiqueta}'] = porcentaje
            except Exception as e:
                print(f"    Error clasificación: {e}")
                for etiqueta in ['Estable','Ganancia_Veg','Perdida_Veg','Disturbio']:
                    stats_cambios[f'Pct_{etiqueta}'] = 0.0

        # ---- Área
        try:
            area_ha = aoi_geom.area().getInfo() / 10000.0
        except:
            area_ha = 0.0

        resultado = {'AOI': aoi_nombre, 'Area_ha': area_ha, **stats_periodo, **stats_cambios}
        resultados.append(resultado)
        print(f"  ✓ {aoi_nombre} procesado")

    return resultados

# (Re)calcular estadísticas y refrescar tabla_final si quieres NBR y dNBR en tablas
estadisticas_aoi = extraer_estadisticas_por_aoi(composiciones, cambios_temporales, aois)
interpretaciones = interpretar_cambios_completos(estadisticas_aoi)
tabla_final = crear_tabla_final(estadisticas_aoi, interpretaciones)

print("✓ Función extraer_estadisticas_por_aoi actualizada con NBR/dNBR")

=== EXTRAYENDO ESTADÍSTICAS POR AOI (NDVI+NBR) ===
Procesando: Punto 24 (k12+100)
  ✓ Punto 24 (k12+100) procesado
Procesando: Punto 23 (k12+040)
  ✓ Punto 23 (k12+040) procesado
Procesando: Punto 22 (k11+950)
  ✓ Punto 22 (k11+950) procesado
Procesando: Punto 21  (k11+710)
  ✓ Punto 21  (k11+710) procesado
Procesando: Punto 20 (k11+600)
  ✓ Punto 20 (k11+600) procesado
Procesando: Punto 18 (k11+330)
  ✓ Punto 18 (k11+330) procesado
Procesando: Punto 17 (k11+150)
  ✓ Punto 17 (k11+150) procesado
Procesando: Punto 16 (k10+850)
  ✓ Punto 16 (k10+850) procesado
Procesando: Punto 15 (k10+730)
  ✓ Punto 15 (k10+730) procesado
Procesando: Punto 14 (k10+490)
  ✓ Punto 14 (k10+490) procesado
Procesando: Punto 13 (k10+340)
  ✓ Punto 13 (k10+340) procesado
Procesando: Punto 11 (k8+900)
  ✓ Punto 11 (k8+900) procesado
Procesando: Punto 12 (k9+100)
  ✓ Punto 12 (k9+100) procesado
Procesando: Punto 10 (k8+440)
  ✓ Punto 10 (k8+440) procesado
Procesando: Punto 9 (7+170)
  ✓ Punto 9 (7+170) procesado

In [18]:
# -----------------------------------------------------------------------------
# (B) Helpers para reshapes y nombres de períodos
# -----------------------------------------------------------------------------
def _tidy_metric(df, prefix):
    """
    Convierte columnas tipo f'{prefix}_1990s', f'{prefix}_2000s', ... a formato long
    """
    cols = [c for c in df.columns if c.startswith(prefix + '_')]
    dfl = df[['AOI'] + cols].melt(id_vars='AOI', value_vars=cols, var_name='Variable', value_name=prefix)
    dfl['Periodo'] = dfl['Variable'].str.replace(prefix + '_', '', regex=False)
    dfl.drop(columns=['Variable'], inplace=True)
    return dfl


def _order_periodo_label(s, style="compact_multiline"):
    """
    Devuelve (categorical_ordenado, dict_etiquetas) para mapear a ejes.
    style: 'long', 'compact', 'compact_multiline'
    """
    orden = ['1990s', '2000s', '2010s', 'L8_extendido', 'S2_extendido', 'fusion_extendido']
    cat = pd.Categorical(s, categories=orden, ordered=True)

    etiquetas_long = {
        '1990s': '1990s (L5)',
        '2000s': '2000s (L7)',
        '2010s': '2010s (L8)',
        'L8_extendido': 'Presente (L8)',
        'S2_extendido': 'Presente (S2)',
        'fusion_extendido': 'Presente (L8+S2)'
    }
    etiquetas_compact = {
        '1990s': '90s (L5)',
        '2000s': '00s (L7)',
        '2010s': '10s (L8)',
        'L8_extendido': 'Pres. (L8)',
        'S2_extendido': 'Pres. (S2)',
        'fusion_extendido': 'Pres.\n(L8+S2)'  # 2 líneas
    }
    if style == "long":
        etiquetas = etiquetas_long
    elif style == "compact":
        etiquetas = etiquetas_compact
    else:  # compact_multiline
        etiquetas = {k: v.replace(" (", "\n(") if "(" in v else v for k, v in etiquetas_compact.items()}
        etiquetas['fusion_extendido'] = 'Presente\n(L8+S2)'

    return cat, etiquetas

# -----------------------------------------------------------------------------
# (C) Gráficas: series temporales NDVI/NBR
# -----------------------------------------------------------------------------
def graficar_series_temporales(tabla, metrica='NDVI', por_aoi=False):
    dfl = _tidy_metric(tabla, metrica)
    dfl = dfl.dropna(subset=[metrica])
    cat, etiquetas = _order_periodo_label(dfl['Periodo'])
    dfl['Periodo'] = cat
    dfl['Etiqueta'] = dfl['Periodo'].astype(str).map(etiquetas)

    if por_aoi:
        # 1 figura por AOI
        for aoi, sub in dfl.groupby('AOI'):
            plt.figure(figsize=(9,5))
            plt.plot(sub['Etiqueta'], sub[metrica], marker='o')
            plt.title(f'{metrica} medio por período — {aoi}')
            plt.xlabel('Período')
            plt.ylabel(metrica)
            plt.ylim(-0.3, 1.0 if metrica=='NDVI' else 1.0)
            plt.tight_layout()
            out = GRAF_DIR / f"serie_{metrica}_{aoi}.png"
            plt.savefig(out, dpi=200)
            plt.close()
    else:
        # Todos los AOI en una sola (líneas por AOI)
        plt.figure(figsize=(11,6))
        for aoi, sub in dfl.groupby('AOI'):
            plt.plot(sub['Etiqueta'], sub[metrica], marker='o', label=aoi)
        plt.title(f'{metrica} medio por período — Todos los AOI')
        plt.xlabel('Período'); plt.ylabel(metrica)
        plt.legend(ncol=2, fontsize=9)
        plt.ylim(-0.3, 1.0 if metrica=='NDVI' else 1.0)
        plt.tight_layout()
        out = GRAF_DIR / f"serie_{metrica}_todos.png"
        plt.savefig(out, dpi=200)
        plt.close()

# -----------------------------------------------------------------------------
# (D) Gráficas: barras apiladas de porcentajes de cambio
# -----------------------------------------------------------------------------
def graficar_porcentajes_apilados(tabla):
    cols = [c for c in tabla.columns if c.startswith('Pct_')]
    if not cols:
        print("No hay columnas Pct_* en tabla_final. Omite esta gráfica.")
        return
    dfp = tabla[['AOI'] + cols].copy()
    # Normaliza por fila (por si no suma 100 por redondeos/faltantes)
    dfp[cols] = dfp[cols].fillna(0)
    row_sums = dfp[cols].sum(axis=1)
    row_sums[row_sums == 0] = 1.0
    dfp[cols] = dfp[cols].div(row_sums, axis=0) * 100

    order = ['Pct_Estable','Pct_Ganancia_Veg','Pct_Perdida_Veg','Pct_Disturbio']
    dfp = dfp[['AOI'] + [c for c in order if c in dfp.columns]]
    dfp = dfp.set_index('AOI')

    plt.figure(figsize=(12,6))
    bottom = np.zeros(len(dfp))
    for c in dfp.columns:
        plt.bar(dfp.index, dfp[c].values, bottom=bottom, label=c.replace('Pct_',''))
        bottom += dfp[c].values
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('% de área'); plt.title('Clasificación de cambios (apilado por AOI)')
    plt.legend(title='Clase')
    plt.tight_layout()
    out = GRAF_DIR / "porcentajes_cambio_apilado.png"
    plt.savefig(out, dpi=200)
    plt.close()

def preparar_df_periodos(df, col_periodo='periodo', style='compact_multiline'):
    cat, etiquetas = _order_periodo_label(df[col_periodo], style=style)
    df = df.assign(periodo_cat=cat).dropna(subset=['periodo_cat']).sort_values('periodo_cat')
    df['label'] = df[col_periodo].map(etiquetas)
    return df, etiquetas
def plot_linea_periodos(df, ycol, titulo, ax=None, rotate=0):
    df, _ = preparar_df_periodos(df, style='compact_multiline')
    if ax is None:
        # ancho dinámico en función del # de etiquetas
        w = max(6, 1.6 * df['label'].nunique())
        fig, ax = plt.subplots(figsize=(w, 3.2), constrained_layout=True, dpi=150)
    ax.plot(df['label'], df[ycol], marker='o', linewidth=1.5)
    ax.set_title(titulo, fontsize=10)
    ax.set_xlabel("")
    ax.grid(True, axis='y', alpha=0.25)
    ax.tick_params(axis='x', labelsize=9, rotation=rotate)
    ax.margins(x=0.05)
    return ax

print("✓ Funciones de gráficas de series temporales y porcentajes definidas")

✓ Funciones de gráficas de series temporales y porcentajes definidas


In [19]:
# -----------------------------------------------------------------------------
# (E) Gráficas: dNDVI y Magnitud por comparación (barras)
# -----------------------------------------------------------------------------
def graficar_cambios_barras(tabla, metrica_prefijo='dNDVI_'):
    # junta columnas dNDVI_* en long
    cols = [c for c in tabla.columns if c.startswith(metrica_prefijo)]
    if not cols:
        print(f"No hay {metrica_prefijo}* en tabla_final.")
        return
    dfl = tabla[['AOI'] + cols].melt(id_vars='AOI', value_name='valor', var_name='Cambio')
    dfl['Cambio'] = dfl['Cambio'].str.replace(metrica_prefijo, '', regex=False)

    plt.figure(figsize=(12,6))
    sns.barplot(data=dfl, x='AOI', y='valor', hue='Cambio')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel(metrica_prefijo.rstrip('_'))
    plt.title(f'{metrica_prefijo.rstrip("_")} por comparación y AOI')
    plt.axhline(0, color='k', linewidth=0.8)
    plt.tight_layout()
    out = GRAF_DIR / f"{metrica_prefijo.rstrip('_')}_por_comparacion.png"
    plt.savefig(out, dpi=200)
    plt.close()

# -----------------------------------------------------------------------------
# (F) Láminas PDF por AOI (4 paneles)
# -----------------------------------------------------------------------------
def exportar_pdf_laminas(tabla):
    pdf_path = GRAF_DIR / "laminas_multiespectral_por_AOI.pdf"
    with PdfPages(pdf_path) as pdf:
        # preparar tidy NDVI y NBR
        ndvi = _tidy_metric(tabla, 'NDVI').dropna()
        nbr  = _tidy_metric(tabla, 'NBR').dropna()
        ndvi['Periodo'], lbl = _order_periodo_label(ndvi['Periodo'])
        nbr['Periodo'],  _   = _order_periodo_label(nbr['Periodo'])

        # dNDVI y Magnitud en long
        d_cols  = [c for c in tabla.columns if c.startswith('dNDVI_')]
        m_cols  = [c for c in tabla.columns if c.startswith('Magnitud_')]
        dn = tabla[['AOI'] + d_cols].melt(id_vars='AOI', var_name='Comp', value_name='dNDVI') if d_cols else None
        mg = tabla[['AOI'] + m_cols].melt(id_vars='AOI', var_name='Comp', value_name='Magnitud') if m_cols else None
        if dn is not None: dn['Comp'] = dn['Comp'].str.replace('dNDVI_','', regex=False)
        if mg is not None: mg['Comp'] = mg['Comp'].str.replace('Magnitud_','', regex=False)

        for aoi in tabla['AOI']:
            fig = plt.figure(figsize=(14,9))
            gs = fig.add_gridspec(2,2)

            # (1) Serie NDVI
            ax1 = fig.add_subplot(gs[0,0])
            sub = ndvi[ndvi['AOI']==aoi].copy()
            if len(sub):
                sub['Etiqueta'] = sub['Periodo'].astype(str).map(lbl)
                ax1.plot(sub['Etiqueta'], sub['NDVI'], marker='o')
            ax1.set_title(f"{aoi} — NDVI medio por período")
            ax1.set_ylim(-0.3,1.0); ax1.set_xlabel(''); ax1.set_ylabel('NDVI')

            # (2) Serie NBR
            ax2 = fig.add_subplot(gs[0,1])
            sub2 = nbr[nbr['AOI']==aoi].copy()
            if len(sub2):
                sub2['Etiqueta'] = sub2['Periodo'].astype(str).map(lbl)
                ax2.plot(sub2['Etiqueta'], sub2['NBR'], marker='o')
            ax2.set_title(f"{aoi} — NBR medio por período")
            ax2.set_ylim(-1.0,1.0); ax2.set_xlabel(''); ax2.set_ylabel('NBR')

            # (3) dNDVI por comparación
            ax3 = fig.add_subplot(gs[1,0])
            if dn is not None:
                sub3 = dn[dn['AOI']==aoi].copy()
                sns.barplot(data=sub3, x='Comp', y='dNDVI', ax=ax3)
                ax3.axhline(0, color='k', linewidth=0.8)
                ax3.set_title("dNDVI por comparación"); ax3.set_xlabel(''); ax3.set_ylabel('dNDVI')
                for tick in ax3.get_xticklabels(): tick.set_rotation(20)

            # (4) Porcentajes apilados
            ax4 = fig.add_subplot(gs[1,1])
            pct_cols = [c for c in tabla.columns if c.startswith('Pct_')]
            if pct_cols:
                fila = tabla[tabla['AOI']==aoi][pct_cols].fillna(0.0)
                labels = [c.replace('Pct_','') for c in pct_cols]
                vals   = fila.values.flatten()
                # normaliza a 100
                s = vals.sum(); vals = (vals/s*100) if s>0 else vals
                ax4.bar(labels, vals)
                ax4.set_ylim(0,100); ax4.set_title("Clasificación de cambio (%)")
                for tick in ax4.get_xticklabels(): tick.set_rotation(20)

            fig.suptitle(f"Lámina multiespectral — {aoi}", fontsize=16, y=0.98)
            plt.tight_layout()
            pdf.savefig(fig, dpi=250)
            plt.close(fig)
    print(f"📄 PDF generado: {pdf_path}")

print("✓ Funciones de gráficas de cambios y PDF por AOI definidas")

✓ Funciones de gráficas de cambios y PDF por AOI definidas


In [20]:
# -----------------------------------------------------------------------------
# (G) Ejecución de las gráficas
# -----------------------------------------------------------------------------
graficar_series_temporales(tabla_final, metrica='NDVI', por_aoi=False)
graficar_series_temporales(tabla_final, metrica='NBR',  por_aoi=False)
graficar_series_temporales(tabla_final, metrica='NDVI', por_aoi=True)  # fichas por AOI
graficar_porcentajes_apilados(tabla_final)
graficar_cambios_barras(tabla_final, metrica_prefijo='dNDVI_')
graficar_cambios_barras(tabla_final, metrica_prefijo='Magnitud_')
exportar_pdf_laminas(tabla_final)

print("✅ Gráficas generadas en:", GRAF_DIR)

📄 PDF generado: salidas_aoi/graficas/laminas_multiespectral_por_AOI.pdf
✅ Gráficas generadas en: salidas_aoi/graficas
